<a href="https://colab.research.google.com/github/karinadw/Neutrino-classification-CNN/blob/main/classifying_neutrinos_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLASSIYING NEUTRINOS IN A LONG BASE-LINE NEUTRINO OSCILLATION EXPERIMENT

## Karina Dansinghani Wadhwani

## Student number: 18014458 

## Date: January 2021

The code to conduct this project was discussed with student Teresa Delgado de las Heras with student number 18079736.

# *INDEX*

1. [Introduction](#intro)

2. [Background and theory](#back)

3. [Task 1: developing a classifier to identify $\nu_\mu$ CC events](#task1)

  3.1 [Retrieving the data](#data)

  3.2 [Binary classification](#binary)

  3.3 [Balancing the data](#balance)

  3.4 [Preparing the data](#prep)

  3.5 [Multi-view Convolutional Neural Network](#CNN)

4. [Task 2: investigating how the variables in the meta data affect the accuracy of the classifier](#task2)

  4.1 [How to define the performance of the classifier?](#4.1)

  4.2 [How well does the classifier perform on QE events vs DIS events?](#4.2)

  4.3 [How well does the classifier perform on low energy neutrinos vs high energy neutrinos?](#4.3)

  4.4 [How well does the classifier perform on low energy muons vs high energy muons?](#4.4)

  4.5 [Which variables in the metadata does the classifier performance depend?](#4.5)

5. [Extension 1: Machine learning algorithm to determine the energy of the neutrino](#extension1)

6. [Extension 2: Machine learning algorithm to determine the flavour of the neutrino](#extension2)

7. [Extension 3: Machine learning algorithm to determine $y=$ lepton energy over neutrino energy](#extension3)

8. [Extension 5: Machine learning algorithm to determine the interaction mode](#extension5)

8. [Conclusion](#conc)

9. [References](#ref)

<a name="intro"></a>

## 1. Introduction

In the following project meta-data on neutrino events is going to be used to identify $\nu_\mu$ charged current events as well as to explore how the different variable of the meta data affect the results obtained. Some other classification tasks being done include a interaction mode classifier as well as a neutrino flavour classifier. Some regression tasks being done in the project are neutrino energy detection and lepton energy over neutrino energy detection. 

All of these are common tasks tackled within the computer science field of computer vision. 

<a name="back"></a>

## 2. Background and theory

Particles in the Standard Model are classified into two categories: fermions and boson. All the particles in the Standard Model have an antiparticle, which has the same spin and mass as the particle but has opposite charge. Bosons have integer spin and are force carriers. Fermions have half-integer spin and are grouped into quarks and leptons. An example of a fermion is the electron. Muon (µ) and tau ($\tau$) electrons are the two heavier counterparts of the electrons, which form the charged leptons along with their antiparticles. In a weak interaction, these charged leptons can couple to a neutrino forming the electron neutrino $\nu_{\mu}$, muon neutrino $\nu_{e}$, and tau neutrino $\nu_{\tau}$. These three particles are types of neutrinos, also known as flavours.  

The only neutrino interactions with matter that have been observed are mediated by the weak force through the exchange of either the $Z^0$ or $W^+$ bosons. Neutral current (NC) events are those interactions that are mediated by the $Z^0$ as it is neutral, while charged current (CC) events are mediated by the $W^+$ boson. Due to charge conservation, in an NC interaction the particle is left with its charge unchanged, therefore an interacting neutrino remains a neutrino. On the other hand, CC interactions undergo a change in charge, the interaction neutrino will transform into its lepton partner, leaving a signature of the flavour of the incoming neutrino. In terms of the nuclear side of the interaction, this can be a lot more complex. The simplest case occurs when the neutrino interacts without significant coupling to the nucleus with a single nucleon. This is considered an elastic interaction for NC events, while the CC events are referred to as a quasi-elastic interaction (QE) because the nucleon alters its form. Resonant production (RES) are those events in which a neutrino interaction may excite a resonance within the nucleus to create a hadron. Lastly, more complex scattering events are named deep-inelastic scattering (DIS), these produce significant disruption in the nuclear system.

<a name="task1"></a>

## 3. Task 1: developing a classifier to identify $\nu_\mu$ CC events

The first part of this project consists of developing a machine learning classifier that will succesfully classify $\nu_\mu$ charged-current events. 


Image classification is a popular task tackled in the computer vision field of computer science, where the aim is to teach computers how to "see" as humans do. Recent emerging techonologies and advancement in machine learning has enabled this field to progress significantly with the breakthrough of Convolutional Neural Networks (CNN), inspired by animal visual cortex perception system. This is a deep neural network that is going to be used throughout the project for different tasks. 

For this specific task, a CNN is going to be used for a classification problem.
The meta data available for this task contains information on the neutrino interaction and images of the x-z view and y-z view for each interaction. These will be used to train a multi-view CNN. 



The first step is to import all the necessary modules. 

In [ ]:
## IMPORTING ALL THE NECESSARY MODULES

import matplotlib.pyplot as plt
import numpy as np
import math
import h5py

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate, AveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import callbacks 

import matplotlib.style                     
import matplotlib as mpl        

# Necessary to retrieve the data
import urllib.request   

# Used to split data into training and testing data samples
from sklearn.model_selection import train_test_split

# Imports a progress bar
from tqdm.notebook import tqdm              
from IPython.display import display, Math

# To be able to save the files from google colaboratory to my desktop 
from google.colab import files              

# Set default figure size
mpl.rcParams["legend.frameon"] = False
mpl.rcParams['figure.dpi'] = 200            # dots per inch

# Useful for debugging problems
print("Tensorflow version: ", tf.__version__)

In [ ]:
import enum 
class Interaction(enum.Enum):
    kNumuQE =0            # Numu CC QE interaction
    kNumuRes =1           # Numu CC Resonant interaction
    kNumuDIS = 2          # Numu CC DIS interaction
    kNumuOther = 3        # Numu CC, other than above
    kNueQE = 4            # Nue CC QE interaction
    kNueRes = 5           # Nue CC Resonant interaction
    kNueDIS = 6           # Nue CC DIS interaction
    kNueOther = 7         # Nue CC, other than above
    kNutauQE = 8          # Nutau CC QE interaction
    kNutauRes = 9         # Nutau CC Resonant interaction
    kNutauDIS =10         # Nutau CC DIS interaction
    kNutauOther =11       # Nutau CC, other than above
    kNuElectronElastic = 12# NC Nu On E Scattering
    kNC =13               # NC interaction
    kCosmic =14           # Cosmic ray background
    kOther =15            # Something else.  Tau?  Hopefully we don't use this
    kNIntType=16          # Number of interaction types, used like a vector size

    
class FinalState(enum.Enum):
    kNumu0tr0sh=0                    # Numu CC - no track no shower
    kNumu0tr1sh=1                    # Numu CC - no track  1 shower
    kNumu0tr2sh=enum.auto()          # Numu CC - no track  2 shower
    kNumu0trMsh=enum.auto()          # Numu CC - no track 3+ shower
    kNumu1tr0sh=enum.auto()          # Numu CC -  1 track no shower
    kNumu1tr1sh=enum.auto()          # Numu CC -  1 track  1 shower
    kNumu1tr2sh=enum.auto()          # Numu CC -  1 track  2 shower
    kNumu1trMsh=enum.auto()          # Numu CC -  1 track 3+ shower
    kNumu2tr0sh=enum.auto()          # Numu CC -  2 track no shower
    kNumu2tr1sh=enum.auto()          # Numu CC -  2 track  1 shower
    kNumu2tr2sh=enum.auto()          # Numu CC -  2 track  2 shower
    kNumu2trMsh=enum.auto()          # Numu CC -  2 track 3+ shower
    kNumuMtr0sh=enum.auto()          # Numu CC - 3+ track no shower
    kNumuMtr1sh=enum.auto()          # Numu CC - 3+ track  1 shower
    kNumuMtr2sh=enum.auto()          # Numu CC - 3+ track  2 showe
    kNumuMtrMsh=enum.auto()          # Numu CC - 3+ track 3+ shower
    kNue0tr0sh=enum.auto()           # Nue CC - no track no shower
    kNue0tr1sh=enum.auto()           # Nue CC - no track  1 shower
    kNue0tr2sh=enum.auto()           # Nue CC - no track  2 shower
    kNue0trMsh=enum.auto()           # Nue CC - no track 3+ shower
    kNue1tr0sh=enum.auto()           # Nue CC -  1 track no shower
    kNue1tr1sh=enum.auto()           # Nue CC -  1 track  1 shower
    kNue1tr2sh=enum.auto()           # Nue CC -  1 track  2 shower
    kNue1trMsh=enum.auto()           # Nue CC -  1 track 3+ shower
    kNue2tr0sh=enum.auto()           # Nue CC -  2 track no shower
    kNue2tr1sh=enum.auto()           # Nue CC -  2 track  1 shower
    kNue2tr2sh=enum.auto()           # Nue CC -  2 track  2 shower
    kNue2trMsh=enum.auto()           # Nue CC -  2 track 3+ shower
    kNueMtr0sh=enum.auto()           # Nue CC - 3+ track no shower
    kNueMtr1sh=enum.auto()           # Nue CC - 3+ track  1 shower
    kNueMtr2sh=enum.auto()           # Nue CC - 3+ track  2 shower
    kNueMtrMsh=enum.auto()           # Nue CC - 3+ track 3+ shower
    kNC0tr0sh=enum.auto()            # NC CC - no track no shower
    kNC0tr1sh=enum.auto()            # NC CC - no track  1 shower
    kNC0tr2sh=enum.auto()            # NC CC - no track  2 shower
    kNC0trMsh=enum.auto()            # NC CC - no track 3+ shower
    kNC1tr0sh=enum.auto()            # NC CC -  1 track no shower
    kNC1tr1sh=enum.auto()            # NC CC -  1 track  1 shower
    kNC1tr2sh=enum.auto()            # NC CC -  1 track  2 shower
    kNC1trMsh=enum.auto()            # NC CC -  1 track 3+ shower
    kNC2tr0sh=enum.auto()            # NC CC -  2 track no shower
    kNC2tr1sh=enum.auto()            # NC CC -  2 track  1 shower
    kNC2tr2sh=enum.auto()            # NC CC -  2 track  2 shower
    kNC2trMsh=enum.auto()            # NC CC -  2 track 3+ shower
    kNCMtr0sh=enum.auto()            # NC CC - 3+ track no shower
    kNCMtr1sh=enum.auto()            # NC CC - 3+ track  1 shower
    kNCMtr2sh=enum.auto()            # NC CC - 3+ track  2 shower
    kNCMtrMsh=enum.auto()            # NC CC - 3+ track 3+ shower
    kCosmicFS=enum.auto()            # Cosmic ray background
    kOtherFS=enum.auto()             # Something else.  Tau?  Hopefully we don't use this
    kNFStType=enum.auto()            # Number of interaction types, used like a vector size



<a name="data"></a>

## 3.1 Retrieving the data

There are 200 data files containing neutrino events in it. As these files are going to be used throughout the project and, in ocasions, more or less files may be needed, it is crucial to create a definition to retrieve as many data files needed. 

In [ ]:
## CREATING A FUNCTION TO RETRIEVE THE DATA FILES

def data_retriever(file_number):
  """
  This function takes the number of files to be retrieved as an input and 
  retrieves the data and returns the file names in an array.  

  Input:
  - file_number: number of files to be retrieved.

  Output:
  - f_name = an array with the name of the files corresponding 
  to the number of files requested

  """
  # the link to the files we want to retrieve look like this:
  # http://www.hep.ucl.ac.uk/undergrad/0056/other/projects/nova/neutrino1.h5
  # the number after neutrino is the only thing that changes for the 200 files, 
  # a counter will be used to iterate over the different links

  core = 'http://www.hep.ucl.ac.uk/undergrad/0056/other/projects/nova/neutrino'
  counter = 1
  end = '.h5'
  f_name=[]  # empty array to store the name of the files being retreived 
  
  # iterating over the number of files requested 
  for i in tqdm(range(0,file_number)):

          filename = "neutrino" + str(counter) + ".h5"                # obtaining the name of the link of the file 
          f_name.append(filename)                                     # appending the name to the array of filenames 
          urllib.request.urlretrieve(core+str(counter)+end, filename) # retriving the data of the files
          counter += 1                                                # increasing the counter by 1 
          
  return (f_name)

Now that we have a definition to retrieve the data, we can proceed to use it. After testing the performance of the network with several files, I have decided to use 7 files as using more did not change the performance significantly and it is computationally a lot more expensive. 

In [ ]:
## RETRIEVING THE FILES FOR TASK 1 

files = data_retriever(25)

## 3.2 Binary classification 

In order to succesfully detect $\nu_\mu$ charged-current events, a binary classification has to be done. Binary classification is the process in which elements are groupes into two different sets on the basis of some classification rule. In this case the two groups are 0s and 1s, where the 1 represents the $\nu_\mu$ charged-current events and the 0 represents everything else. 


In [ ]:
## CHANGING LABELS FOR A BINARY CLASSIFICATION, 1 REPRESENTS MUON NEUTRINO CC EVENTS AND 0 REPRESENTS EVERYTHING ELSE. 

model_lab_imb = []                                                # empty array for the labels of the neutrino interactions, either 0s or 1s
model_in_1_imb = []                                               # empty array for the x-z view of the neutrino interaction 
model_in_2_imb = []                                               # empty array for the y-z view of the neutrino interaction

# iterating over the array of file names to obtain the lables and the input images for each interaction
for file_name in tqdm(files):

    # opening the file in read only mode 
    df=h5py.File(file_name, 'r')
    
    # iterating over the length of cvnmap to obtain the labels and input images for each interaction 
    for i in range(len(df['cvnmap'])):
      model = df['cvnmap'][i].reshape((2,100,80))                 # reshaping the images 
      model_in_1_imb.append(model[0])                             # apending the x-z view to the corresponding array
      model_in_2_imb.append(model[1])                             # apending the y-z view to the corresponding array

      if df['neutrino']['interaction'][i] <= 3:
        model_lab_imb.append(int(1))                              # appending a 1 to the labels for muon neutrino CC events 

      else:
        model_lab_imb.append(int(0))                              # appending a 0 to the labels for non muon neutrino CC events

We are now going to plot the labels in a histogram. 

In [ ]:
# PLOTTING LABELS FOR BINARY CLASSIFICATION IN A HISTOGRAM

plt.hist(model_lab_imb)                                       # calls the labels of the model 
plt.title('Labels for muon neutrino classification')          # adds a title 
plt.ylabel('Number of neutrino intercations')                 # adds a y label to the histogram
plt.xlabel('Label')                                           # adds a y label to the histogram

It can clearly be seen from the graph above that there is a significant data imbalance. This is due to the nature of these events. Detection of muon neutrino events is a lot higher than electron neutrino events. On the other hand, tau neutrino events have never even been detected, their existence has been infered. 

The percentage of each type of event in the loaded data files can be seen below. 

In [ ]:
numu_counter = 0               # counter to keep track of how many muon neutrino CC events there are 
nue_counter = 0                # counter to keep track of how many electron neutrino CC events there are
nutau_counter = 0              # counter to keep track of how many tau neutrino CC events there are
other_counter = 0              # counter to keep track of how many other events there are

# iterating over the loaded data files to obtain the number of each type of events 
for file_name in tqdm(files):

  # opening the file in read mode only 
  df=h5py.File(file_name, 'r')

  # iterating over the length of neutrino interactions in the data file 
  for i in range(len(df['cvnmap'])):

    # if the neutrino interaction number is between 0 and/or equal to 3, 
    # the counter for muon neutrino events is increased by 1
    if df['neutrino']['interaction'][i] <= 3:
      numu_counter= numu_counter + 1

    # if the neutrino interaction number is between 4 and/or equal to 7, 
    # the counter for electron neutrino events is increased by 1
    elif df['neutrino']['interaction'][i] > 4 and df['neutrino']['interaction'][i] <= 7:
      nue_counter= nue_counter + 1

    # if the neutrino interaction number is between 7 and/or equal to 11, 
    # the counter for tau neutrino events is increased by 1
    elif df['neutrino']['interaction'][i] > 7 and df['neutrino']['interaction'][i] <= 11:

      nutau_counter= nutau_counter + 1

    # for all other events the counter for other events is increased by 1 
    else:

      other_counter = other_counter + 1
      
# the total events adds all the events to get the total value of events in the data files 
totalevents = numu_counter + nue_counter + nutau_counter + other_counter

In [ ]:
## DISPLAYING THE NUMBER OF EVENTS AND THEIR CORRESPONDING PERCENTAGES 

# Muon neutrino charged-current events information 
display(Math(r'There \; are \; {} \; \nu_\mu \; charged-current \; events. \; This \; represents \; {} \% \; of \; the \; events \; in \; {} \; meta \; data \; files.'.format(numu_counter, round(((numu_counter / totalevents ) * 100),2), len(files))))

# Tau neutrino charged-current events information 
display(Math(r'There \; are \; {} \; \nu_\tau \; charged-current \; events. \; This \; represents \; {} \% \; of \; the \; events \; in \; {} \; meta \; data \; files.'.format(nutau_counter, round(((nutau_counter / totalevents ) * 100),2), len(files))))

# Electron neutrino charged-current events information 
display(Math(r'There \; are \; {} \; \nu_e \; charged-current \; events. \; This \; represents \; {} \% \; of \; the \; events \; in \; {} \; meta \; data \; files.'.format(nue_counter, round(((nue_counter / totalevents ) * 100),2), len(files))))

# Other events information 
display(Math(r'There \; are \; {} \; other \; events. \; This \; represents \; {} \% \; of \; the \; events \; in \; {} \; meta \; data \; files.'.format(other_counter, round(((other_counter / totalevents ) * 100),2), len(files))))

Having this type of imabalance in the data set can lead to the accuracy paradox, where if we tried to classify, for example, electron neutrino CC events we would end up getting a higher accuracy in our model than for muon neutrino CC events, despite having more data for the latter. It is therefore important to make sure the dataset is balanced before proceeding. 

<a name="balance"></a>

## 3.3 Balancing the data

The method used to balance the data is as follows. All the data files are going to be loaded, then they are going to split into files that are going to be used to obtain muon neutrino charged-current events and files that will be used to obtain the data for non-muon neutrino charged-current events. As it can be seen from section 3.2, the imbalance is significant, there are a lot more muon netrino interactions. For this reason, more files are going to be used to iterate and get muon neutrino charged-current events and less files are going to be used for non-muon neutrino charged current events. 

After testing out different percentages, it was found that using 100% for the rest yielded in a balanced dataset. 

In [ ]:
## SPLITTING THE DATA FILES 

# using 100% of the files for non muon neutrino CC events and 10% for the rest
files_90, files_10 = train_test_split(files, train_size = 0.9, shuffle = False)

Now that the files have been split, the same methodology as in section 3.2 will be used to label the events. 

In [ ]:
## BINARY CLASSIFICATION AND BALANCING THE DATA 
# using 1s to label muon neutrino CC events and 0s to label the rest

# empty arrays
model_lab=[]                                                     # empty array for the labels of the neutrino interactions, either 0s or 1s 
model_in_1 = []                                                  # empty array for the x-z view of the neutrino interaction
model_in_2 = []                                                  # empty array for the y-z view of the neutrino interaction


# iterating over the array of file names to obtain the lables and the input images for each interaction
for file_name in tqdm(files):

    # opening the file in read only mode 
    df=h5py.File(file_name, 'r')
    
    # iterating over the length of cvnmap to obtain the labels and input images for each interaction 
    for i in range(len(df['cvnmap'])):
      
      # if the interaction number is above 3 it is a non neutrino CC event
      if df['neutrino']['interaction'][i] > 3:

        model = df['cvnmap'][i].reshape((2,100,80))                 # reshaping the images 
        model_in_1.append(model[0])                                 # apending the x-z view to the corresponding array
        model_in_2.append(model[1])                                 # apending the y-z view to the corresponding array
        model_lab.append(int(0))                                    # appends a 0 to the labels array for non muon neutrino CC interactions

      else:
        pass


# iterating over the array of file names to obtain the lables and the input images for each interaction
for file_name in tqdm(files_10):

    # opening the file in read only mode 
    df=h5py.File(file_name, 'r')
    
    # iterating over the length of cvnmap to obtain the labels and input images for each interaction 
    for j in range(len(df['cvnmap'])):
      
      # if the interaction number is below 3 it is a neutrino CC event
      if df['neutrino']['interaction'][j] <= 3:

        model = df['cvnmap'][j].reshape((2,100,80))                 # reshaping the images 
        model_in_1.append(model[0])                                 # apending the x-z view to the corresponding array
        model_in_2.append(model[1])                                 # apending the y-z view to the corresponding array
        model_lab.append(int(1))                                    # appends a 1 to the labels array for muon neutrino CC interactions

      else:
        pass

The balanced data set is going to be plotted in a histogram along with the imabalanced data set to check if it has been correctly balanced. 

In [ ]:
## HISTOGRAMS OF BALANCED AND IMBALANCED DATASETS 

f = plt.figure(figsize=(20,7))           # sets the size of the histograms 
ax = f.add_subplot(121)                  # adds one plot 
ax2 = f.add_subplot(122)                 # adds a second plot next to the first one 
ax.hist(model_lab_imb)                   # the first histogram is the imbalanced labels 
ax.set_title('Imbalanced data set')      # adds a title to the histogram on the right 
ax2.hist(model_lab)                      # the first histogram is the balanced labels
ax2.set_title('Balanced data set')       # adds a title to the histogram on the left
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it


<a name="prep"></a>

## 3.4 Preparing the data

Now that we have the labels, it is important to shuffle them as we have all the 0s at the beggining and all the 1s at the end of the array. In order to shuffle the labels, we must shuffle the images as well so that they are all shuffled together and the information is not mixed up. To do so, and as this will be necessary throughout the project, a function will be created. 

In [ ]:
## FUNCTION TO SHUFFLE THE IMAGES AND LABELS TOGETHER 

def shuffle_data(labels, image_set_1, image_set_2):  
  
  """
  This function takes the labels and the two sets of corresponding images as an input and shuffles them together.  

  Input:
  - labels: labels of the neutrino interactions.
  - image_set_1: images corresponding to the x-z view of the interaction.
  - image_set_2: images corresponding to the y-z view of the interaction.

  Output:
  - labels: shuffled labels of the neutrino interactions.
  - image_set_1: shuffled images corresponding to the x-z view of the interaction.
  - image_set_2: shuffled images corresponding to the y-z view of the interaction.

  """

  zip_data = list(zip(labels, image_set_1, image_set_2))    # zips all the data to be able to shuffle it together 
  np.random.shuffle(zip_data)                               # shuffled the data 
  labels, image_set_1, image_set_2 = zip(*zip_data)         # unzips the data 

  return labels, image_set_1, image_set_2

The data has to be prepared in order to feed it into the network. As this is a procedure that will have to be done throughout the project, a function will be defined. In this function, the labels and the two input images will be put into arrays. These will then be split into data to train the model and data to test the model. The training data will further be split into training and validation data. Lastly, the data will be normalized so that it is between 0 and 1 pixels and it will be converted to the same data type. 

In [ ]:
def data (model_input_1, model_input_2, model_labels, train_frac, val_frac):

  """
  This function takes the labels and the two sets of corresponding images as an input and puts them into separate arrays,
  splits the data into training, testing and validation data sets, normalizes the images to be between 0 and 1 pixels and 
  puts all the data in the same format. It prepares the data to be fed into the neural network. 

  Input:
  - model_labels: labels of the neutrino interactions.
  - model_input_1: images corresponding to the x-z view of the interaction.
  - model_input_2: images corresponding to the y-z view of the interaction.
  - train_frac: fraction to split the data into training and testing data. If 80% is used, 80% will be used for training and 20% for testing. 
  - val_fraction: fraction to split the training data into training and validation data. 

  Output:
  - train_input_1: training data corresponding to the x-z view of the interaction. 
  - test_input_1: testing data corresponding to the x-z view of the interaction. 
  - val_input_1: validation data corresponding to the x-z view of the interaction. 
  - train_input_2: training data corresponding to the y-z view of the interaction. 
  - test_input_2: testing data corresponding to the y-z view of the interaction. 
  - val_input_2: validation data corresponding to the y-z view of the interaction. 
  - train_labels: training labels of the interaction. 
  - val_labels: validation labels of the interaction.  
  - test_labels: testing labels of the interaction. 

  """

  # Making arrays of all the data 
  model_in_1 = np.array(model_input_1)
  model_in_2 = np.array(model_input_2)
  model_lab = np.array(model_labels)
  
  # Splitting the data into train and test data
  tr_input_1, te_input_1 = train_test_split(model_in_1, train_size = train_frac, shuffle = False)
  tr_input_2, te_input_2 = train_test_split(model_in_2, train_size = train_frac, shuffle = False)
  tr_labels, te_labels = train_test_split(model_lab, train_size = train_frac, shuffle = False)

  # Splitting the data into training and validation data sets
  train_input_1, val_input_1 = train_test_split(tr_input_1, train_size = val_frac, shuffle = False)
  train_input_2, val_input_2 = train_test_split(tr_input_2, train_size = val_frac, shuffle = False)
  train_labels, val_labels =train_test_split(tr_labels, train_size = val_frac, shuffle = False)

  # Normalizing the images to be between 0 and 1 pixels and converting the arrays to the same data type  
  train_input_1 = train_input_1.astype('float32') / 255.0
  val_input_1 = val_input_1.astype('float32') / 255.0
  test_input_1 = te_input_1.astype('float32') / 255.0

  train_input_2 = train_input_2.astype('float32') / 255.0
  val_input_2 = val_input_2.astype('float32') / 255.0
  test_input_2 = te_input_2.astype('float32') / 255.0

  train_labels = train_labels.astype('float32')
  val_labels = val_labels.astype('float32')
  test_labels = te_labels.astype('float32')

  return (train_input_1, test_input_1, val_input_1, train_input_2, test_input_2, val_input_2, train_labels, val_labels, test_labels)


Having these definitions we are going to prepare the data. First we will shuffle all the images and labels, then we will expand the dimensions as we need a shape of 4 for the images of the neutrino interactions to feed into the neural network and lastly we will put the data into arrays and split it to obtain training, testing and validation data. 

In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# shuffling the labels and images 
model_lab, model_in_1, model_in_2 = shuffle_data(model_lab, model_in_1, model_in_2)

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
model_in_1 = tf.expand_dims(model_in_1, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
model_in_2 = tf.expand_dims(model_in_2, axis = 3)

# preparing the training, testing and validation data
train_input_1, test_input_1, val_input_1, train_input_2, test_input_2, val_input_2, train_labels, val_labels, test_labels = data(model_in_1, model_in_2, model_lab, 0.8, 0.8)

<a name="CNN"></a>

## 3.5 Multi-view Convolutional Neural Network

The inputs from the x-y view and the y-z are handled separately in two branches of the network and are then merged in the final few layers of the model, this is called a multi-view CNN. Where the network is learning about one neutrino interaction by “viewing” it from different perspectives. This multi-view CNN model is going to be created using the keras functional API which can handle models with non-linear topology, shared layers, and even multiple inputs or outputs. The architecture of this network, which can be seen in Figure 4, was inspired by VGGNet and ResNet but was adapted to this specific task. 


The model contains 3 convolutional layers for feature extraction for both the inputs. Convolution1D is usually used for input signals similar to the voice, Convolution2D is used for images and Convolution3D is used for videos. Therefore, Convolution2D is the first hidden layer being used. The second hidden layer is MaxPooling2D, which downsamples the inputs by calculating the maximum value in each patch of every feature map. AveragePooling2D could be used but for computer visions tasks such as this one, MaxPooling2D has been proved to be more effective. Lastly a Dropout layer is used for regularization technique to avoid overfitting. This drops a percentage of data randomly. 





In [ ]:
def create_convolution_layers(input_img, input_shape):

  """
  This function creates a simple Convolutional Neural Network model in keras using the functional API.

  Input:
  - input_img: image to be fed into the network, either the x-z view of the interaction or the y-z view.
  - input_shape: shape of the images being fed into the network.

  Output:
  - model: CNN model  

  """
  # releases the global state and helps avoid clutter from other models
  #keras.backend.clear_session   

  # Conv2D layer is used for images.  
  model = Conv2D(8, (3, 3),                                                     # Using a filter size of 8, kernel size of (3,3)
                 activation = "relu",                                           # Rectified Linear Unit as activation function
                 padding='same', input_shape=input_shape[1:])(input_img)        # Keeping the padding 'same' allows to preserve spatial dimension
  # Max pooling operation for 2D spatial data.
  model = MaxPooling2D((2, 2),                                                  # Pool size of (2,2)
                       padding='same')(model)                                   # Keeping the padding 'same' allows to preserve spatial dimension
  # Regularization technique for the model
  model = Dropout(0.25)(model)                                                  # 25% of the data is randomly excluded 
  
  # Conv2D layer is used for images.
  model = Conv2D(16, (3, 3),                                                    # Using a filter size of 16, kernel size of (3,3)
                 padding='same')(model)                                         # Keeping the padding 'same' allows to preserve spatial dimension
  # Max pooling operation for 2D spatial data.
  model = MaxPooling2D((2, 2),                                                  # Pool size of (2,2)
                       padding='same')(model)                                   # Keeping the padding 'same' allows to preserve spatial dimension
  model = Dropout(0.25)(model)                                                  # 25% of the data is randomly excluded 
 
  # Conv2D layer is used for images.
  model = Conv2D(16, (3, 3),                                                    # Using a filter size of 32, kernel size of (3,3)
                 padding='same')(model)                                         # Keeping the padding 'same' allows to preserve spatial dimension
  # Max pooling operation for 2D spatial data.
  model = MaxPooling2D((2, 2),                                                  # Pool size of (2,2)
                       padding='same')(model)                                   # Keeping the padding 'same' allows to preserve spatial dimension
  model = Dropout(0.4)(model)                                                   # 40% of the data is randomly excluded    
 
  return model

 The two inputs are merged by a concatenate layer and then go through a Flatten layer that converts the 2D matrix into a vector. It then goes through a Dense layer which connects all the neuron in a layer to the neurons in the next layer, a ReLU activation function and a Dropout layer. At last, it goes through another Dense layer and a sigmoid activation function, which transforms the input into a value between 0.0 and 1.0. 

In [ ]:
def concatenating (model_input_1):

  """
  This function creates a simple Convolutional Neural Network model in keras using the functional API for both the views of the 
  neutrino intercations, for the x-y view and the y-z view, and concatenates them. 

  Input:
  - model_input_1: this is the image of the x-z view of the neutrino interaction. It is used to obtain the shape of the model input.

  Output:
  - model: CNN model of concatenated x-z and y-z views of the neutrino interactions.

  """

  # Creates the CNN model for the x-z view of the neutrino interaction.
  xz_input = Input(shape=np.shape(model_input_1)[1:])
  xz_model = create_convolution_layers(xz_input, np.shape(model_input_1)[1:])

  # Creates the CNN model for the y-z view of the neutrino interaction.
  yz_input = Input(shape=np.shape(model_input_1)[1:])
  yz_model = create_convolution_layers(yz_input, np.shape(model_input_1)[1:])

  # Concatenates the two models 
  conv = concatenate([xz_model, yz_model])

  # Flattens the concatenated models 
  conv = Flatten()(conv)

  # Dense layers connect all the neurons of one layer to the ones in the next layer 
  dense = Dense(8,                                                              # 64 is the dimensionality of the output space
                activation = "relu")(conv)                                      # Rectified Linear Unit as activation function
  dense = Dropout(0.5)(dense)                                                   # 50% of the data is randomly excluded                                                   # 50% of the data is randomly excluded 

  output = Dense(1,                                                             # 1 is the dimensionality of the output space 
                 activation ="sigmoid")(dense)                                  # sigmoid as activation function. 
                                                                                # The input to the function is transformed into a value between 0.0 and 1.0.
 
  # creates the model using the two model inputs 
  model = Model(inputs = [xz_input, yz_input], outputs = [output])

  # compiles the final model
  model.compile(loss=tf.keras.losses.binary_crossentropy,                       # using binary crossentropy as a loss function as we are doing a binary classification
  # adam is used for optimization algorithm for stochastic gradient descent for training deep learning models
                optimizer='adam',                                               
                metrics=['accuracy'])                                           # using the accuracy as a metric to evaluate the model 

  model.summary()

  return model 

Now that we have all the definitions, we can create the model. 

In [ ]:
## CREATING THE MODEL FOR MUON NEUTRINO CLASSIFICATION

model = concatenating(model_in_1)

Now that the model has been created, it can be trained on the training dataset and evaluated on the valuation dataset. Callbacks.EarlyStopping is used to avoid overtraining. 

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history = model.fit(x=[train_input_1,train_input_2],y=train_labels, batch_size=64, # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_input_1,val_input_2],val_labels), 
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

Now that the model has been trained, the accuracy and loss is evaluated and plotted. 

In [ ]:
test_loss, test_acc = model.evaluate([test_input_1,test_input_2],  test_labels, verbose=2)

In [ ]:
## PLOTTING THE ACCURACY OF THE MODEL 

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid()
plt.title('Accuracy of the muon neutrino classifier')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## PLOTTING THE LOSS OF THE MODEL 

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid()
plt.title('Loss of muon neutrino classifier')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## MODEL PREDICTIONS 

predictions = model.predict([test_input_1,test_input_2])

plt.hist(predictions,bins=50) # plots the predictions in a histogram 

<a name="task2"></a>

## 4. Task 2: investigating how the variables in the meta data affect the accuracy of the classifier

In this section we are going to investigate how some of the variables in the meta data affects the performance of the classifier. 

<a name="4.1"></a>

## 4.1 How to define the performance of the classifier?

I am going to test the clasifier to detect electron neutrinos with the original meta data. As there is approximately 2% electron neutrino charged-current events in the meta data available, we would expect the accuracy for this classifier to be lower than the classifier of muon neutrinos. By according to the accuracy paradox, we will obtain a higher accuracy if we use the imbalanced dataset. This means we would obtain false-positives. We are going to test the accuracy paradox to evaluate how important it is to balance the data. 




For this, we firstly need to import the files from which we will obtain the data. This time we will label the electron neutrino charged-current events as 1s and the rest as 0s. 

In [ ]:
## RETRIEVING THE DATA FILES FOR TASK 2

files_t2 = data_retriever(5)

In [ ]:
## CHANGING LABELS FOR A BINARY CLASSIFICATION, 1 REPRESENTS ELECTRON NEUTRINO CC EVENTS AND 0 REPRESENTS EVERYTHING ELSE. 

model_lab_elec = []                                                # empty array for the labels of the neutrino interactions, either 0s or 1s
model_in_1_elec = []                                               # empty array for the x-z view of the neutrino interaction 
model_in_2_elec = []                                               # empty array for the y-z view of the neutrino interaction

# iterating over the array of file names to obtain the lables and the input images for each interaction
for file_name in tqdm(files_t2):

    # opening the file in read only mode 
    df=h5py.File(file_name, 'r')
    
    # iterating over the length of cvnmap to obtain the labels and input images for each interaction 
    for i in range(len(df['cvnmap'])):
      model = df['cvnmap'][i].reshape((2,100,80))                  # reshaping the images 
      model_in_1_elec.append(model[0])                             # apending the x-z view to the corresponding array
      model_in_2_elec.append(model[1])                             # apending the y-z view to the corresponding array

      if df['neutrino']['interaction'][i] > 3 and df['neutrino']['interaction'][i] <= 7:
        model_lab_elec.append(int(1))                              # appending a 1 to the labels for electron neutrino CC events 

      else:
        model_lab_elec.append(int(0))                              # appending a 0 to the labels for non electron neutrino CC events


In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# shuffling the labels and images 
model_lab_elec, model_in_1_elec, model_in_2_elec = shuffle_data(model_lab_elec, model_in_1_elec, model_in_2_elec)

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
model_in_1_elec = tf.expand_dims(model_in_1_elec, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
model_in_2_elec = tf.expand_dims(model_in_2_elec, axis = 3)

# preparing the training, testing and validation data
train_input_1_elec, test_input_1_elec, val_input_1_elec, train_input_2_elec, test_input_2_elec, val_input_2_elec, train_labels_elec, val_labels_elec, test_labels_elec = data(model_in_1_elec, model_in_2_elec, model_lab_elec, 0.8, 0.8)

In [ ]:
## CREATING THE MODEL FOR ELECTRON NEUTRINO CLASSIFICATION

model_elec = concatenating(model_in_1_elec)

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history_elec = model_elec.fit(x=[train_input_1_elec,train_input_2_elec],
                    y=train_labels_elec, batch_size=128,                           # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_input_1_elec,val_input_2_elec],val_labels_elec), 
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

In [ ]:
test_loss, test_acc = model_elec.evaluate([test_input_1_elec,test_input_2_elec],  test_labels_elec, verbose=2)

In [ ]:
## PLOTTING THE ACCURACY OF THE MODEL 


plt.plot(history_elec.history['accuracy'], label='accuracy')
plt.plot(history_elec.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper right')
plt.grid()
plt.title('Accuracy of electron neutrino classifier')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## PLOTTING THE LOSS OF THE MODEL 

plt.plot(history_elec.history['loss'], label='loss')
plt.plot(history_elec.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid()
plt.title('Loss of electron neutrino classifier')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

A higher accuracy is obtained for classification of electron neutrino charged-current events, as expected from the accuracy paradox. Although it can be seen from the graph of the accuracy that the model is not really learning. This demonstrated the importance of having a balanced dataset. 

To ensure this observation is correct, this classification will be performed on a balanced dataset. 

In [ ]:
## RETRIEVING THE FILES FOR ELECTRON NEUTRINO CLASSIFICATION 

files_t2_2 = np.array(data_retriever(68))

To balance the data for electron neutrino CC events we will need to iterate through significantly more files to obtain the data corresponding to electron neutrinos and very few for the rest. 

Here we are using 70 files to obtain the data corresponding to electron neutrinos and 1 files for non-electron neutrino CC events, this represents 1% of the datafiles. 

In [ ]:
## SPLITTING THE DATA FILES 

# using 100% of the files for electron neutrino CC events and 1% for the rest
files_t2_2_90, files_t2_2_10 = train_test_split(files_t2_2, train_size = 0.99)

In [ ]:
## LABELS FOR BINARY CLASSIFICATION AND BALANCING THE DATA 
# using 1s to label electron neutrino CC events and 0s to label the rest

# empty arrays
model_lab_elec_b=[]                                                     # empty array for the labels of the neutrino interactions, either 0s or 1s 
model_in_1_elec_b = []                                                  # empty array for the x-z view of the neutrino interaction
model_in_2_elec_b = []                                                  # empty array for the y-z view of the neutrino interaction


# iterating over the array of file names to obtain the lables and the input images for each interaction
for file_name in tqdm(files_t2_2):

    # opening the file in read only mode 
    df=h5py.File(file_name, 'r')
    
    # iterating over the length of cvnmap to obtain the labels and input images for each interaction 
    for i in range(len(df['cvnmap'])):
      
      # if the interaction number is above between 4 and 7 it is an electron neutrino CC event
      if (df['neutrino']['interaction'][i])>=4 and (df['neutrino']['interaction'][i])<=7: 

        model = df['cvnmap'][i].reshape((2,100,80))                     # reshaping the images 
        model_in_1_elec_b.append(model[0])                              # apending the x-z view to the corresponding array
        model_in_2_elec_b.append(model[1])                              # apending the y-z view to the corresponding array
        model_lab_elec_b.append(int(1))                                 # appends a 1 to the labels array for electron neutrino CC interactions

      else:
        pass


# iterating over the array of file names to obtain the lables and the input images for each interaction
for file_name in tqdm(files_t2_2_10):

    # opening the file in read only mode 
    df=h5py.File(file_name, 'r')
    
    # iterating over the length of cvnmap to obtain the labels and input images for each interaction 
    for j in range(len(df['cvnmap'])):
      
      # if the interaction number is below 3 or above 8 it is not an electron neutrino CC event
      if (df['neutrino']['interaction'][j])<=3 or (df['neutrino']['interaction'][j])>=8:

        model = df['cvnmap'][j].reshape((2,100,80))                     # reshaping the images 
        model_in_1_elec_b.append(model[0])                              # apending the x-z view to the corresponding array
        model_in_2_elec_b.append(model[1])                              # apending the y-z view to the corresponding array
        model_lab_elec_b.append(int(0))                                 # appends a 1 to the labels array for muon neutrino CC interactions

      else:
        pass

Now that we have the labels, we should plot it on a histogram to make sure the data is balanced. 


In [ ]:
## HISTOGRAM OF LABELS FOR ELECTRON NEUTRINO CLASSIFICATION 

plt.hist(model_lab_elec_b)
plt.title('Balanced data set for electron neutrino CC events detection')

In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# shuffling the labels and images 
model_lab_elec_b, model_in_1_elec_b, model_in_2_elec_b = shuffle_data(model_lab_elec_b, model_in_1_elec_b, model_in_2_elec_b)

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
model_in_1_elec_b = tf.expand_dims(model_in_1_elec_b, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
model_in_2_elec_b = tf.expand_dims(model_in_2_elec_b, axis = 3)

# preparing the training, testing and validation data
train_input_1_elec_b, test_input_1_elec_b, val_input_1_elec_b, train_input_2_elec_b, test_input_2_elec_b, val_input_2_elec_b, train_labels_elec_b, val_labels_elec_b, test_labels_elec_b = data(model_in_1_elec_b, model_in_2_elec_b, model_lab_elec_b, 0.8, 0.8)

In [ ]:
## CREATING THE MODEL FOR ELECTRON NEUTRINO CLASSIFICATION

model_elec_b = concatenating(model_in_1_elec_b)

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history = model_elec_b.fit(x=[train_input_1_elec_b,train_input_2_elec_b],
                    y=train_labels_elec_b, batch_size=32,                          # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_input_1_elec_b,val_input_2_elec_b],val_labels_elec_b),
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

In [ ]:
test_loss, test_acc = model.evaluate([test_input_1_elec_b,test_input_2_elec_b],  test_labels_elec_b, verbose=2)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid()
plt.title('Accuracy of model')

It can be seen how the accuracy is reduced once the dataset is balanced. This demonstrates how important it is to balance the data in order to now get false-trues. 

<a name="4.2"></a>

## 4.2 How well does the classifier perform on QE events vs DIS events?

A QE, quasi-elastic, charged-current events usually only has two track and is therefore a "clean" events. On the other hand, DIS, deep inelastic scattering, charged-current events are a a lot more "messy" as it can have several tracks and showers. In this section we are going to explore how do these type of events affect the results of the classifier. 

We will start by checking how many QE and DIS CC events there were in task 1 and then exploring the other options. 

In [ ]:
## RETRIEVING THE FILES FOR TASK 4.2

files_t2_3 = np.array(data_retriever(20))

In [ ]:
## CHECKING HOW MANY QE AND DIS EVENTS ARE THERE IN THE DATASET

QE_counter = 0                                                  # counter to keep track of how many QE events there are 
DIS_counter = 0                                                 # counter to keep track of how many DIS events there are 
total_events = 0                                                # counter to keep track of the total number of interactions taking place

# iterating over the loaded data files to obtain the number of each type of events 
for file_name in tqdm(files_t2_3):

  # opening the file in read mode only 
  df=h5py.File(file_name, 'r')

  # total number of events in that file 
  tot = len(df['neutrino']['interaction'])

  # adds the number of events of that file to the global total number of events 
  total_events = total_events + tot
  
  # iterating over the length of the images of neutrino interactions in the data file
  for i in range(len(df['cvnmap'])):  

    # if the neutrino interaction is equal to 0, 4 or 8 it is QE CC events
    if (df['neutrino']['interaction'][i]) == 0 or (df['neutrino']['interaction'][i]) == 4 or (df['neutrino']['interaction'][i]) == 8: 

      QE_counter = QE_counter + 1 
    
    # if the neutrino interaction is equal to 2, 6 or 19 it is DIS CC events
    elif (df['neutrino']['interaction'][i]) == 2 or (df['neutrino']['interaction'][i]) == 6 or (df['neutrino']['interaction'][i]) == 10:
      
      DIS_counter = DIS_counter + 1

In [ ]:
## CALCULATING THE PERCENTAGE OF QE AND DIS CC EVENTS IN THE DATASET 

percent_DIS = "{:.2f}".format((DIS_counter / total_events)*100)                 # calculates the percentage of DIS CC events in the data 
percent_QE = "{:.2f}".format((QE_counter / total_events)*100)                   # calculates the percentage of QE CC events in the data 

# prints the number of QE and DIS events and their corresponding percentages
print ("Number of files used:",len(files_t2_3), "\nTotal number of events:", total_events, "\nNumber of CC DIS events:", DIS_counter, 
       ". This represents a ", percent_DIS, "% of the total events", "\nNumber of CC QE events: ", QE_counter, 
       ". This represents a", percent_QE,"% of the total events")


From the results above we can see that for 20 files there is a higher percentage of CC DIS events compared to QE events. There is approximately 54% CC DIS events in the file while there is approximately 12% CC QE events. It was found that these percentage were approximately held for all the files. Therefore, the accuracy obtained for the classifier in task 1 is for a dataset with a higher number of CC DIS events compared to CC QE events. Which means the classifier has been learning more on "messier" events. As this is an image classification task I would expect the classifier to perform better on CC QE, "neater", events. We are going to test this in this section. We will first test the performance of the classifier in a dataset with equal number of CC QE and CC DIS events and then on a dataset with a higher number of CC QE events. 

### Performance for equal number of CC QE and CC DIS events 

The first thing we need to do is balance the labels and the CC QE and CC DIS events. For this we will need to iterate on different number of files. 

In [ ]:
## SPLITTING THE DATA TO BALANCE THE LABELS AND CC DIS AND CC QE EVENTS 

files_80, files_20 = train_test_split(files_t2_3, train_size = 0.7, shuffle = False)
files_80_2, files_20_2 = train_test_split(files_20, train_size = 0.7, shuffle = False)

In [ ]:
## LABELS FOR BINARY CLASSIFICATION OF MUON NEUTRINO CC EVENTS 
## AND BALANCING THE CC QE AND CC DIS EVENTS IN MUON NEUTRINO INTERACTIONS

model_in_1_bal_even = []                                                # empty array for the x-z view of the neutrino interaction
model_in_2_bal_even = []                                                # empty array for the y-z view of the neutrino interaction
model_lab_bal_even = []                                                 # empty array for the labels of the neutrino interactions, either 0s or 1s 
QE_counter = 0                                                          # counter to keep track of how many QE events there are 
DIS_counter = 0                                                         # counter to keep track of how many DIS events there are 

# iterating over the array of file names to obtain the lables and the input images for each interaction
for f_name in tqdm(files_t2_3):                                         # using 20 files 

  # opening the file in read only mode
  df=h5py.File(f_name, 'r')

  # iterating over the length of cvnmap to obtain the labels and input images for each interaction 
  for i in range(len(df['cvnmap'])):

    # if the neutrino interaction number is above 3 it is a non-muon neutrino interaction
    if df['neutrino']['interaction'][i] > 3:

        model = (df['cvnmap'][i]).reshape((2,100,80))                   # reshaping the images 
        model_in_1_bal_even.append(model[0])                            # apending the x-z view to the corresponding array
        model_in_2_bal_even.append(model[1])                            # apending the y-z view to the corresponding array
        model_lab_bal_even.append(int(0))                               # assigns a 0 to non muon neutrino CC events  

    else:
      pass 

# iterating over the array of file names to obtain the lables and the input images for each interaction
for f_name in tqdm(files_20):                                           # using 4 files 

  # opening the file in read only mode
  df=h5py.File(f_name, 'r')

  # iterating over the length of cvnmap to obtain the labels and input images for each interaction 
  for j in range(len(df['cvnmap'])):

    # if the interaction number is 0 it is a muon neutrino CC QE event 
    if df['neutrino']['interaction'][j] == 0:

      model = (df['cvnmap'][j]).reshape((2,100,80))                       # reshaping the images 
      model_in_1_bal_even.append(model[0])                                # apending the x-z view to the corresponding array
      model_in_2_bal_even.append(model[1])                                # apending the y-z view to the corresponding array
      model_lab_bal_even.append(int(1))                                   # assigns 1 to muon neutrino CC events 

      QE_counter = QE_counter + 1

    else:
      pass 

# iterating over the array of file names to obtain the lables and the input images for each interaction
for f_name in tqdm(files_20_2):                                           # using 1 files 

  # opening the file in read only mode
  df=h5py.File(f_name, 'r')

  # iterating over the length of cvnmap to obtain the labels and input images for each interaction
  for z in range(len(df['cvnmap'])):

    # if the neutrino interaction number is equal or below 3 it is a muon neutrino CC event
    if df['neutrino']['interaction'][z] <= 3:
      
      model = (df['cvnmap'][z]).reshape((2,100,80))                       # reshaping the images 
      model_in_1_bal_even.append(model[0])                                # apending the x-z view to the corresponding array
      model_in_2_bal_even.append(model[1])                                # apending the y-z view to the corresponding array
      model_lab_bal_even.append(int(1))                                   # assigns 1 to muon neutrino CC events 

      # is the neutrino interaction number is 0 it is a QE event        
      if (df['neutrino']['interaction'][z]) == 0: 
        QE_counter = QE_counter + 1 

      # is the neutrino interaction number is 0 it is a DIS event    
      elif (df['neutrino']['interaction'][z]) == 2:
      
        DIS_counter = DIS_counter + 1

    else:
      pass 

In [ ]:
## CHECKING IF QE CC EVENTS AND DIS CC EVENTS ARE BALANCED

print("Muon neutrino CC QE events:", QE_counter)
print("Muon neutrino CC DIS events:", DIS_counter)

In [ ]:
## CHECKING IF THE LABELS ARE BALANCED 

plt.hist(model_lab_bal_even)
plt.title('Labels for muon neutrino classification on a QE and DIS balanced dataset')

In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# shuffling the labels and images 
model_lab_bal_even, model_in_1_bal_even, model_in_2_bal_even = shuffle_data(model_lab_bal_even, model_in_1_bal_even, model_in_2_bal_even)

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
model_in_1_bal_even = tf.expand_dims(model_in_1_bal_even, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
model_in_2_bal_even = tf.expand_dims(model_in_2_bal_even, axis = 3)

# preparing the training, testing and validation data
train_input_1, test_input_1, val_input_1, train_input_2, test_input_2, val_input_2, train_labels, val_labels, test_labels = data(model_in_1_bal_even, model_in_2_bal_even, model_lab_bal_even,0.8,0.8)

In [ ]:
## CREATING THE MODEL FOR MUON NEUTRINO CLASSIFICATION

model = concatenating(model_in_1_bal_even)

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history = model.fit(x=[train_input_1,train_input_2],
                    y=train_labels, batch_size=64,                                 # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_input_1,val_input_2],val_labels),
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

In [ ]:
## EVALUATING THE ACCURACY AND LOSS OF THE MODEL 

test_loss, test_acc = model.evaluate([test_input_1,test_input_2],  test_labels, verbose=2)

In [ ]:
## PLOTTING THE ACCURACY 

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid()
plt.title('Accuracy of muon neutrino classifier for \n a dataset with equal QE and DIS events')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## PLOTTING THE LOSS 

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid()
plt.title('Loss of muon neutrino classifier for \n a dataset with equal QE and DIS events')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

It can clearly be seen that the accuracy increase when the classifier operates on a dataset with equal number of CC QE and CC DIS events, as expected. 

We are now going to test the classifier on a dataset of more CC QE than CC DIS events, it is expected that the accuracy should increase even more having "clean" events. 

### PERFORMANCE OF THE CLASSIFIER ON DATASET WITH ONLY MUON NEUTRINO CC QE EVENTS

The muon neutrino classifier is going to be tested on a dataset which only has CC QE events. 

In [ ]:
files_80, files_20 = train_test_split(files_t2_3, train_size = 0.8, shuffle = False)
files_80_2, files_20_2 = train_test_split(files_20, train_size = 0.7, shuffle = False)

In [ ]:
## LABELS FOR BINARY CLASSIFICATION AND BALANCING THE DATA 
# using 1s to label electron neutrino CC events and 0s to label the rest
# only using CC QE events 

model_in_1_qe = []                                                          # empty array for the labels of the neutrino interactions, either 0s or 1s 
model_in_2_qe = []                                                          # empty array for the x-z view of the neutrino interaction
model_lab_qe = []                                                           # empty array for the y-z view of the neutrino interaction
QE_counter_qe = 0                                                           # counter to keep track of how many CC QE events there are 
DIS_counter_qe = 0                                                          # counter to keep track of how many CC DIS events there are 

# iterating over the loaded data files to obtain the number of each type of events 
for f_name in tqdm(files_t2_3):                                             # using 20 files 

  # opening the file in read mode only 
  df=h5py.File(f_name, 'r')

  # iterating over the length of neutrino interactions in the data file
  for i in range(len(df['cvnmap'])):

    # if the neutrino interaction number is higher than 3, it is a non-muon neutrino event
    if df['neutrino']['interaction'][i] > 3:
      
        model = (df['cvnmap'][i]).reshape((2,100,80))                       # reshaping the images 
        model_in_1_qe.append(model[0])                                      # apending the x-z view to the corresponding array
        model_in_2_qe.append(model[1])                                      # apending the y-z view to the corresponding array
        model_lab_qe.append(int(0))                                         # assigns a 0 to non muon neutrino CC events  
        
    else:
      pass 

# iterating over the loaded data files to obtain the number of each type of events 
for f_name in tqdm(files_t2_3):                                             # using 4 files 

  # opening the file in read mode only 
  df=h5py.File(f_name, 'r')

  # iterating over the length of images in the data file
  for j in range(len(df['cvnmap'])):

    # is the neutrino interaction corresponds to 0 it is a muon neutrino CC QE event
    if df['neutrino']['interaction'][j] == 0:

      model = (df['cvnmap'][j]).reshape((2,100,80))                       # reshaping the images
      model_in_1_qe.append(model[0])                                      # apending the x-z view to the corresponding array
      model_in_2_qe.append(model[1])                                      # apending the y-z view to the corresponding array
      model_lab_qe.append(int(1))                                         # assigns 1 to muon neutrino CC events 

      QE_counter_qe = QE_counter_qe + 1

    else:
      pass 

In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# shuffling the labels and images 
model_lab_qe, model_in_1_qe, model_in_2_qe = shuffle_data(model_lab_qe, model_in_1_qe, model_in_2_qe)

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
model_in_1_qe = tf.expand_dims(model_in_1_qe, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
model_in_2_qe = tf.expand_dims(model_in_2_qe, axis = 3)

# preparing the training, testing and validation data
train_input_1_qe, test_input_1_qe, val_input_1_qe, train_input_2_qe, test_input_2_qe, val_input_2_qe, train_labels_qe, val_labels_qe, test_labels_qe = data(model_in_1_qe, model_in_2_qe, model_lab_qe, 0.8, 0.8)

In [ ]:
## PLOTTING A HISTOGRAM OF THE LABELS TO CHECK IF THEY ARE BALANCED 

plt.hist(model_lab_qe)
plt.title('Labels for muon neutrino classification with a dataset of more CC QE events')

In [ ]:
## CREATING THE MODEL FOR ELECTRON NEUTRINO CLASSIFICATION

model = concatenating(model_in_1_qe)

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history_elec = model.fit(x=[train_input_1_qe,train_input_2_qe],
                    y=train_labels_qe, batch_size=64,                              # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_input_1_qe,val_input_2_qe],val_labels_qe),
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

In [ ]:
## EVALUATING THE ACCURACY AND LOSS OF THE MODEL 

test_loss, test_acc = model.evaluate([test_input_1_qe,test_input_2_qe],  test_labels_qe, verbose=2)

In [ ]:
## PLOTTING THE ACCURACY OF THE MODEL 

plt.plot(history_elec.history['accuracy'], label='accuracy')
plt.plot(history_elec.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid()
plt.title('Accuracy of muon neutrino classifier operating \n on a dataset of QE events')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## PLOTTING THE LOSS OF THE MODEL 

plt.plot(history_elec.history['loss'], label='loss')
plt.plot(history_elec.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid()
plt.title('Loss of muon neutrino classifier operating \n on a dataset of QE events')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

## OBSERVATIONS

The highest accuracy for the classifier was obtained for a dataset with only CC QE events, which is what was predicted as these are less "messy" events. 

A good performance was achieved for a dataset of balanced CC QE and CC DIS events.

Lowest performance for a dataset of predominantly CC DIS events, which is what was used for task 1.

<a name="4.3"></a>

## 4.3 How well does the classifier perform on low energy neutrinos vs high energy neutrinos?

In this section we will explore how does the performance of the model change whehn it is operating on high energy neutrino and when it is operating on low energy neutrinos. The thershold for high energy is set at 3.5 MeV, obtained from https://iopscience.iop.org/article/10.1088/1742-6596/718/6/062052/pdf.

From intuition, I am not expecting the accuracies to vary drastically as this is an image classification task and therefore what matters is what is seen on the image. The energy of the neutrinos should not have a significant impact on the clarity of the image.


## Testing performance only on high energy neutrinos

We will start by testing the performance for a dataset that only contains high energy neutrinos. 

In [ ]:
## RETRIEVING THE FILES FOR THIS SECTION 

files_t2_4=data_retriever(15)

In [ ]:
## SPLITTING THE FILES TO BE ABLE TO BALANCE THE LABELS 

files_80, files_20 = train_test_split(files_t2_4, train_size = 0.9, shuffle = False)

In [ ]:
## SETTING THE THRESHOLD FOR HIGH/LOW ENERGY NEUTRINO

threshold = 3.5

In [ ]:
high_e_labels=[]                                                           # empty array to store the labels, 1 for muon neutrino events and 0 for the rest 
high_e_input_1=[]                                                          # empty array for the x-z view of the neutrino interaction
high_e_input_2=[]                                                          # empty array for the y-z view of the neutrino interaction

# iterating over the loaded data files to obtain the number of each type of events 
for filename in tqdm(files_t2_4):

  # opening the file in read mode only
  df=df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions in the data file
  for i in range(len(df['neutrino']['interaction'])):

    # only classifying those neutrino with energy above the threshold 
    if (df['neutrino']['nuenergy'][i]) > threshold:                

      # if the neutrino interaction number is above 3 it is a non-muon neutrino CC event
      if (df['neutrino']['interaction'][i]) > 3: 
        model = (df['cvnmap'][i]).reshape((2,100,80))                     # reshaping the images 
        high_e_input_1.append(model[0])                                   # apending the x-z view to the corresponding array
        high_e_input_2.append(model[1])                                   # apending the y-z view to the corresponding array
        high_e_labels.append(int(0))                                      # 0 for non muon neutrino CC events 

      else:
        pass

# iterating over the loaded data files to obtain the number of each type of events 
for filename in tqdm(files_20):

  # opening the file in read mode only
  df=df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions in the data file
  for j in range(len(df['neutrino']['interaction'])):

    # only classifying those neutrino with energy above the threshold 
    if (df['neutrino']['nuenergy'][j]) > threshold: 

      # if the neutrino interaction number is below or equal to 3 it is a muon neutrino CC event
      if (df['neutrino']['interaction'][j]) <= 3: 

        model = (df['cvnmap'][j]).reshape((2,100,80))                     # reshaping the images 
        high_e_input_1.append(model[0])                                   # apending the x-z view to the corresponding array
        high_e_input_2.append(model[1])                                   # apending the y-z view to the corresponding array
        high_e_labels.append(int(1))                                      # 1 for non muon neutrino CC events 

      else:
        pass

In [ ]:
## CHECKING IF THE LABELS ARE BALANCED 

plt.hist(high_e_labels)
plt.title('Labels for muon neutrino classifier for a dataset of high energy neutrinos')

In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# shuffling the labels and images 
high_e_labels, high_e_input_1, high_e_input_2 = shuffle_data(high_e_labels, high_e_input_1, high_e_input_2)

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
high_e_input_1 = tf.expand_dims(high_e_input_1, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
high_e_input_2 = tf.expand_dims(high_e_input_2, axis = 3)

# preparing the training, testing and validation data
train_input_1, test_input_1, val_input_1, train_input_2, test_input_2, val_input_2, train_labels, val_labels, test_labels = data(high_e_input_1, high_e_input_2, high_e_labels, 0.8, 0.8)

In [ ]:
## CREATING THE MODEL FOR ELECTRON NEUTRINO CLASSIFICATION

model = concatenating(high_e_input_1)

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history = model.fit(x=[train_input_1,train_input_2],
                    y=train_labels, batch_size=64,                                 # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_input_1,val_input_2],val_labels),
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

In [ ]:
## EVALUATING THE ACCURACY AND LOSS OF THE MODEL 

test_loss, test_acc = model.evaluate([test_input_1,test_input_2],  test_labels, verbose=2)

In [ ]:
## PLOTTING THE ACCURACY OF THE MODEL

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid()
plt.title('Accuracy of muon neutrino classifier operating \n on a dataset with high energy neutrinos')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## PLOTTING THE LOSS OF THE MODEL

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid()
plt.title('Loss of muon neutrino classifier operating \n on a dataset with high energy neutrinos')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

## Testing performance only on low energy neutrinos

Now we are going to test the performance of the classifier on a dataset that only contains low energy neutrinos. 

In [ ]:
## RETRIEVING THE NECESSARY FILES FOR THIS TASK 

files_t2_6=data_retriever(30)

In [ ]:
## SPLITTING THE DATA 

files_80_2, files_20_2 = train_test_split(files_t2_6, train_size = 0.9, shuffle = False)

In [ ]:
threshold = 3.5
low_e_labels=[]                                                           # empty array to store the labels, 1 for muon neutrino events and 0 for the rest 
low_e_input_1=[]                                                          # empty array for the x-z view of the neutrino interaction
low_e_input_2=[]                                                          # empty array for the y-z view of the neutrino interaction

# iterating over the loaded data files to obtain the number of each type of events 
for filename in tqdm(files_t2_6):

  # opening the file in read mode only
  df=df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions in the data file
  for i in range(len(df['neutrino']['interaction'])):

    # only classifying those neutrino with energy below the threshold 
    if (df['neutrino']['nuenergy'][i]) < threshold:                

      # if the neutrino interaction number is above 3 it is a non-muon neutrino CC event
      if (df['neutrino']['interaction'][i]) > 3: 
        model = (df['cvnmap'][i]).reshape((2,100,80))                    # reshaping the images 
        low_e_input_1.append(model[0])                                   # apending the x-z view to the corresponding array
        low_e_input_2.append(model[1])                                   # apending the y-z view to the corresponding array
        low_e_labels.append(int(0))                                      # 0 for non muon neutrino CC events 

      else:
        pass

# iterating over the loaded data files to obtain the number of each type of events 
for filename in tqdm(files_20_2):

  # opening the file in read mode only
  df=df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions in the data file
  for j in range(len(df['neutrino']['interaction'])):

    # only classifying those neutrino with energy below the threshold 
    if (df['neutrino']['nuenergy'][j]) < threshold: 

      # if the neutrino interaction number is below or equal to 3 it is a muon neutrino CC event
      if (df['neutrino']['interaction'][j]) <= 3: 

        model = (df['cvnmap'][j]).reshape((2,100,80))                    # reshaping the images 
        low_e_input_1.append(model[0])                                   # apending the x-z view to the corresponding array
        low_e_input_2.append(model[1])                                   # apending the y-z view to the corresponding array
        low_e_labels.append(int(1))                                      # 1 for non muon neutrino CC events 

      else:
        pass

In [ ]:
## PLOTTING THE LABELS TO CHECK IF IT IS BALANCED

plt.hist(low_e_labels)
plt.title('Labels for muon neutrino classification performing on a dataset of low energy neutrinos')

In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# shuffling the labels and images 
low_e_labels, low_e_input_1, low_e_input_2 = shuffle_data(low_e_labels, low_e_input_1, low_e_input_2)

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
low_e_input_1 = tf.expand_dims(low_e_input_1, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
low_e_input_2 = tf.expand_dims(low_e_input_2, axis = 3)

# preparing the training, testing and validation data
train_input_1, test_input_1, val_input_1, train_input_2, test_input_2, val_input_2, train_labels, val_labels, test_labels = data(low_e_input_1, low_e_input_2, low_e_labels, 0.8, 0.8)

In [ ]:
## CREATING THE MODEL FOR ELECTRON NEUTRINO CLASSIFICATION

model = concatenating(low_e_input_1)

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history = model.fit(x=[train_input_1,train_input_2],
                    y=train_labels, batch_size=64,                                 # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_input_1,val_input_2],val_labels),
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

In [ ]:
## EVALUATING THE ACCURACY AND LOSS OF THE MODEL 

test_loss, test_acc = model.evaluate([test_input_1,test_input_2],  test_labels, verbose=2)

In [ ]:
## PLOTTING THE ACCURACY OF THE MODEL

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid()
plt.title('Accuracy of muon neutrino classifier operating \n on a dataset of low energy neutrinos')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## PLOTTING THE LOSS OF THE MODEL

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid()
plt.title('Loss of muon neutrino classifier operating \n on a dataset of low energy neutrinos')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

## Observations:

The accuracy didn't change significantly for a dataset containing more high energy neutrinos or the dataset containing more low energy neutrinos. This verifies our initial expectations. 

<a name="4.4"></a>

## 4.4 How well does the classifier perform on low energy muons vs high energy muons?

We are now going to perform a similar task as above but in this case with high energy muons and low energy muons. I will start by evaluating how many high energy and low energy muons there are in the original dataset. 

In [ ]:
files_t2_7=data_retriever(10)

In [ ]:
threshold = 3.5

In [ ]:
## Counting how many low energy and high energy muons there are in the dataset

low_e_muon = 0                                                    # counter for low energy muon
high_e_muon = 0                                                   # counter for high energy muon 

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_t2_7):

  # opening the file in read only mode
  df=h5py.File(filename, 'r')
 
  # iterating over the length of the neutrino interactions 
  for i in range(len(df['neutrino']['interaction'])):

    if (df['neutrino']['interaction'][i]) <=3:                    # interactions below or equal 3 are muon neutrino CC events

      if (df['neutrino']['nuenergy'][i]) > threshold:
        high_e_muon = high_e_muon + 1 

      else:
        low_e_muon = low_e_muon + 1


In [ ]:
## PRINTING THE NUMBER OF HIGH AND LOW ENERGY MUON EVENTS AND THEIR CORRESPONDING PERCENTAGES

print("Number of high energy muon events:", high_e_muon)
print("Number of low energy muon events:", low_e_muon)

print("Percentage of high energy muons in", len(files_t2_7), "files:", "{:.2f}".format(high_e_muon / (high_e_muon + low_e_muon)*100), "%")
print("Percentage of low energy neutrinos in", len(files_t2_7), "files: " "{:.2f}".format(low_e_muon / (high_e_muon + low_e_muon)*100), "%")

## Observations:

There are more high energy muons than low energy muons in the original datasets, which was used for the classifier in task 1. 

Percentage of high energy muons in 10 files: 64.75 %

Percentage of low energy neutrinos in 10 files: 35.25 %

The classifier is going to be tested on high energy muons and then on low energy muons although no significant change in accuracy is expected, as above. 

### Performance on high energy muons

In [ ]:
## RETRIEVING THE FILES FOR THIS TASK 

files_80_3, files_20_3 = train_test_split(files_t2_7, train_size = 0.8, shuffle = False)

In [ ]:
## LABELS FOR BINARY CLASSIFICATION FOR MUON NEUTRINO CLASSIFICATION
## BEING PERFORMED ON A DATASET OF HIGH ENERGY MUONS 
## BALANCING OUT THE MUON NEUTRINO CC EVENTS AND NON-MUON NEUTRINO CC EVENTS 

high_e_muons_labels = []                                              # empty array for the labels of the neutrino interactions, either 0s or 1s 
high_e_muons_input_1 = []                                             # empty array for the x-z view of the neutrino interaction
high_e_muons_input_2 = []                                             # empty array for the y-z view of the neutrino interaction

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_t2_7):

  # opening the file in read only mode 
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction 
  for i in range(len(df['neutrino']['interaction'])):

    if (df['neutrino']['interaction'][i]) > 3:                        # interactions below or equal 3 are muon neutrino CC events

        model = (df['cvnmap'][i]).reshape((2,100,80))                 # reshaping the images 
        high_e_muons_input_1.append(model[0])                         # apending the x-z view to the corresponding array
        high_e_muons_input_2.append(model[1])                         # apending the y-z view to the corresponding array
        high_e_muons_labels.append(int(0))                            # 0 for non muon neutrino CC events 

    else:
      pass

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_20_3):

  # opening the file in read only mode
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction 
  for i in range(len(df['neutrino']['interaction'])):

    if (df['neutrino']['interaction'][i]) <=3:                        # interactions below or equal 3 are muon neutrino CC events

      # classifying muons with energy above the threshold 
      if (df['neutrino']['nuenergy'][i]) > threshold:

        model = (df['cvnmap'][i]).reshape((2,100,80))                 # reshaping the images 
        high_e_muons_input_1.append(model[0])                         # apending the x-z view to the corresponding array
        high_e_muons_input_2.append(model[1])                         # apending the y-z view to the corresponding array
        high_e_muons_labels.append(int(1))                            # 1 for muon neutrino CC events 

      else:
        pass

In [ ]:
## PLOTTING THE LABELS TO CHECK IF THEY ARE BALANCED 

plt.hist(high_e_muons_labels)
plt.title('Labels for muon neutrino classification on a high energy muon dataset')

In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# shuffling the labels and images 
high_e_muons_labels, high_e_muons_input_1, high_e_muons_input_2 = shuffle_data(high_e_muons_labels, high_e_muons_input_1, high_e_muons_input_2)

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
high_e_muons_input_1 = tf.expand_dims(high_e_muons_input_1, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
high_e_muons_input_2 = tf.expand_dims(high_e_muons_input_2, axis = 3)

# preparing the training, testing and validation data
train_input_1, test_input_1, val_input_1, train_input_2, test_input_2, val_input_2, train_labels, val_labels, test_labels = data(high_e_muons_input_1, high_e_muons_input_2, high_e_muons_labels, 0.8, 0.8)

In [ ]:
## CREATING THE MODEL FOR ELECTRON NEUTRINO CLASSIFICATION

model = concatenating(high_e_muons_input_1)

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history = model.fit(x=[train_input_1,train_input_2],
                    y=train_labels, batch_size=64,                                 # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_input_1,val_input_2],val_labels),
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

In [ ]:
## EVALUATING THE ACCURACY AND LOSS OF THE MODEL 

test_loss, test_acc = model.evaluate([test_input_1,test_input_2],  test_labels, verbose=2)

In [ ]:
## PLOTTING THE ACCURACY OF THE MODEL 

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid()
plt.title('Accuracy of muon neutrino classifier operating \n on a dataset of high energy muons')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## PLOTTING THE LOSS OF THE MODEL 

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid()
plt.title('Loss of muon neutrino classifier operating \n on a dataset of high energy muons')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

## Testing the classifier for low energy muons 

We are now going to repeat the same procedure but for low energy muons instead. 

In [ ]:
## RETRIEVING THE FILES FOR THIS TASK 

files_t2_8=data_retriever(20)

In [ ]:
## SPLITTING THE IMPORTED FILES 

files_80_4, files_20_4 = train_test_split(files_t2_8, train_size = 0.7, shuffle = False)

In [ ]:
## SETTING THE THRESHOLD BETWEEN HIGH AND LOW ENERGY MUONS 

threshold = 3.5

In [ ]:
## LABELS FOR BINARY CLASSIFICATION FOR MUON NEUTRINO CLASSIFICATION
## BEING PERFORMED ON A DATASET OF LOW ENERGY MUONS 
## BALANCING OUT THE MUON NEUTRINO CC EVENTS AND NON-MUON NEUTRINO CC EVENTS 

low_e_muons_labels = []                                              # empty array for the labels of the neutrino interactions, either 0s or 1s 
low_e_muons_input_1 = []                                             # empty array for the x-z view of the neutrino interaction
low_e_muons_input_2 = []                                             # empty array for the y-z view of the neutrino interaction

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_t2_8):

  # opening the file in read only mode 
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction 
  for i in range(len(df['neutrino']['interaction'])):

    if (df['neutrino']['interaction'][i]) > 3:                        # interactions below or equal 3 are muon neutrino CC events

        model = (df['cvnmap'][i]).reshape((2,100,80))                 # reshaping the images 
        low_e_muons_input_1.append(model[0])                          # apending the x-z view to the corresponding array
        low_e_muons_input_2.append(model[1])                         # apending the y-z view to the corresponding array
        low_e_muons_labels.append(int(0))                             # 0 for non muon neutrino CC events 

    else:
      pass

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_20_4):

  # opening the file in read only mode
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction 
  for j in range(len(df['neutrino']['interaction'])):

    if (df['neutrino']['interaction'][j]) <=3:                        # interactions below or equal 3 are muon neutrino CC events

      # classifying muons with energy above the threshold 
      if (df['neutrino']['nuenergy'][j]) < threshold:

        model = (df['cvnmap'][j]).reshape((2,100,80))                 # reshaping the images 
        low_e_muons_input_1.append(model[0])                          # apending the x-z view to the corresponding array
        low_e_muons_input_2.append(model[1])                          # apending the y-z view to the corresponding array
        low_e_muons_labels.append(int(1))                             # 1 for muon neutrino CC events 

      else:
        pass

In [ ]:
## PLOTTING THE LABELS TO CHECK FOR BALANCE 

plt.hist(low_e_muons_labels)
plt.title('Labels for muon neutrino classification on a dataset with low energy muons')

In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# shuffling the labels and images 
low_e_muons_labels, low_e_muons_input_1, low_e_muons_input_2 = shuffle_data(low_e_muons_labels, low_e_muons_input_1, low_e_muons_input_2)

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
low_e_muons_input_1 = tf.expand_dims(low_e_muons_input_1, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
low_e_muons_input_2 = tf.expand_dims(low_e_muons_input_2, axis = 3)

# preparing the training, testing and validation data
train_input_1, test_input_1, val_input_1, train_input_2, test_input_2, val_input_2, train_labels, val_labels, test_labels = data(low_e_muons_input_1, low_e_muons_input_2, low_e_muons_labels, 0.8, 0.8)

In [ ]:
## CREATING THE MODEL FOR MUON NEUTRINO CLASSIFICATION

model = concatenating(low_e_muons_input_1)

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history = model.fit(x=[train_input_1,train_input_2],
                    y=train_labels, batch_size=64,                                 # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_input_1,val_input_2],val_labels),
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

In [ ]:
## EVALUATING THE ACCURACY AND LOSS OF THE MODEL 

test_loss, test_acc = model.evaluate([test_input_1,test_input_2],  test_labels, verbose=2)

In [ ]:
## PLOTTING THE ACCURACY OF THE MODEL 

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid()
plt.title('Accuracy of muon neutrino classifier operating \n on a dataset of low energy muons')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## PLOTTING THE LOSS OF THE MODEL 

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid()
plt.title('Loss of muon neutrino classifier operating \n on a dataset of low energy muons')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

Once again, a similar accuracy is obtained for varying muon energy which is what was expected. 

<a name="4.4"></a>

## 4.5 Which variables in the metadata does the classifier performance depend? 

From the results obtained I think the main variables that affect the performance of the classifier are:

1. Those variables that affect the image being fed into the network. This task is essentially a task within the field of computer vision, where the aim is to replication the human vision system in computers. Therefore those variables that affect what the computer is "seeing" is what will affect the performance of the classifier. 

2. The balance there is in the variables that are affecting the classifier. For example, it is important to have a balance between the labels or a balance between the "messy" and "cleaner" events to obtain accurate results rather than high accuracy results, where we could potentially have flase-trues. 

<a name="extension1"></a>

## 5. Extension 1: Machine learning algorithm to determine the energy of the neutrino.

The value of the energy of the value cannot be classified into categories, we have a range of values and therefore this is a regression task. 

In [ ]:
## RETRIEVING THE FILES FOR THIS EXTENSION

files_e1_2 = data_retriever(5)

Now we need to loop over all the files to get the energy labels and separate the x-z view from the y-z view. 

In [ ]:
## PREPARING THE DATA AND LABELS FOR THE MODEL 

nu_energy_labels=[]                                                      # empty array for the labels of the neutrino energy
nu_energy_input_1=[]                                                     # empty array for the x-z view of the neutrino interaction
nu_energy_input_2=[]                                                     # empty array for the y-z view of the neutrino interaction

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_e1_2):

  # opening the file in read only mode 
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction 
  for i in range(len(df['cvnmap'])):

    model = df['cvnmap'][i].reshape((2,100,80))                           # reshaping the images 
    nu_energy_input_1.append(model[0])                                    # apending the x-z view to the corresponding array
    nu_energy_input_2.append(model[1])                                    # apending the y-z view to the corresponding array
    nu_energy_labels.append(int(df['neutrino']['nuenergy'][i]))           # appending the value of the energy to the labels 

In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
nu_energy_input_1 = tf.expand_dims(nu_energy_input_1, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
nu_energy_input_2 = tf.expand_dims(nu_energy_input_2, axis = 3)

# normalizing the energy labels 
nu_energy_labels=nu_energy_labels/np.max(nu_energy_labels)

# preparing the training, testing and validation data
train_input_1_nuenergy, test_input_1_nuenergy, val_input_1_nuenergy, train_input_2_nuenergy, test_input_2_nuenergy, val_input_2_nuenergy, train_labels_nuenergy, val_labels_nuenergy, test_labels_nuenergy = data(nu_energy_input_1, nu_energy_input_2, nu_energy_labels, 0.8, 0.8)

Now that we have all the data we cab create the model, this uses different activation functions and metrics compared to the classification tasks. We use the mean squared error and mean absolute error as metrics. The optimizer used is the RMSprop algorithm. This maintains a moving (discounted) average of the square of gradients and divides the gradient by the root of this average. For regression tasks, a linear activation function should be used in the last layer. 

In [ ]:
def model_regression(model_input_1):
  
  """
  This function creates a simple Convolutional Neural Network model specifically for regression tasks in keras using the functional API for both the views of the 
  neutrino intercations, for the x-y view and the y-z view, and concatenates them. 

  Input:
  - model_input_1: this is the image of the x-z view of the neutrino interaction. It is used to obtain the shape of the model input.

  Output:
  - model: CNN model of concatenated x-z and y-z views of the neutrino interactions.

  """

  # Creates the CNN model for the x-z view of the neutrino interaction.
  xz_input = Input(shape=np.shape(model_input_1)[1:])
  xz_model = create_convolution_layers(xz_input, np.shape(model_input_1)[1:])

  # Creates the CNN model for the y-z view of the neutrino interaction.
  yz_input = Input(shape=np.shape(model_input_1)[1:])
  yz_model = create_convolution_layers(yz_input, np.shape(model_input_1)[1:])

  # Concatenates the two models 
  conv = concatenate([xz_model, yz_model])

  # Flattens the concatenated models 
  conv = Flatten()(conv)

  # Dense layers connect all the neurons of one layer to the ones in the next layer 
  dense = Dense(8)(conv)                                      
  dense = Dropout(0.5)(dense)                                                   # 50% of the data is randomly excluded                                                   # 50% of the data is randomly excluded 

  output = Dense(1,                                                             # 1 is the dimensionality of the output space 
                  activation ="linear")(dense)                                  # linear as activation function. 
                                                                                # The input to the function is transformed into a value between 0.0 and 1.0.

  # creates the model using the two model inputs 
  model = Model(inputs = [xz_input, yz_input], outputs = [output])

  # compiles the final model
  model.compile(loss='mse',
                optimizer=tf.keras.optimizers.RMSprop(0.001),
                metrics=['mae'                                                # mean absolute error
                , 'mse'])                                                     # mean squared error                                

  model.summary()

  return model 

In [ ]:
## CREATING THE MODEL FOR ENERGY DETERMINATION 

model = model_regression(nu_energy_input_1)

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history_E1 = model.fit(x=[train_input_1_nuenergy,train_input_2_nuenergy],
                    y=train_labels_nuenergy, batch_size=32,                        # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_input_1_nuenergy,val_input_2_nuenergy],
                                     val_labels_nuenergy),
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

In [ ]:
## EVALUATING THE MAE AND MSE OF THE MODEL 

test_loss, test_mae, test_mse = model.evaluate([test_input_1_nuenergy,test_input_2_nuenergy],  test_labels_nuenergy, verbose=2)

In [ ]:
## PLOTTING THE MSE OF THE MODEL 

plt.plot(history_E1.history['mse'], label='mean squared error')
plt.plot(history_E1.history['val_mse'], label = 'validation mean squared error')
plt.xlabel('Epoch')
plt.ylabel('mean squared error')
plt.legend(loc='upper right')
plt.grid()
plt.title('MSE of the model to determine neutrino energy')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## PLOTTING THE MAE OF THE MODEL 

plt.plot(history_E1.history['mae'], label='mean average error')
plt.plot(history_E1.history['val_mae'], label = 'validation mean average error')
plt.xlabel('Epoch')
plt.ylabel('mean average error')
plt.legend(loc='upper right')
plt.grid()
plt.title('MAE of the model to determine neutrino energy')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

<a name="extension2"></a>

## 6. Extension 2: Machine learning algorithm to determine the flavour of the neutrino

This is a similar task as the one performed in task 1 and 2 but instead of having a binary classification, we have more labels that represent the different types of flavours. We will classify these with the number 0, 1 and 2. As there are no tau neutrinos in the data, we can't classify these. 



In [ ]:
## RETRIEVING THE NECESSARY FILES FOR THIS EXTENSION 

files_e2_1=data_retriever(80)

In [ ]:
## SPLITTING THE FILES

files_e2_99, files_e2_01 = train_test_split(files_e2_1, train_size = 0.99)
files_e2_90, files_e2_10 = train_test_split(files_e2_1, train_size = 0.90)

In [ ]:
e2_labels = []                                                              # empty array for the labels of the neutrino flavour
e2_input_1 = []                                                             # empty array for the x-z view of the neutrino interaction
e2_input_2 = []                                                             # empty array for the x-z view of the neutrino interaction

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_e2_01):                                        # using 1 file 

  # opening the file in read only mode
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction 
  for i in range(len(df['neutrino']['interaction'])):

    if df['neutrino']['interaction'][i] <=3:                              # this is for muon neutrinos, between 0-3

      model = df['cvnmap'][i].reshape((2,100,80))                         # reshaping the images
      e2_input_1.append(model[0])                                         # apending the x-z view to the corresponding array
      e2_input_2.append(model[1])                                         # apending the y-z view to the corresponding array
      e2_labels.append(int(1))                                            # appending a 1 to the labels for muon neutrinos 

    else:
      pass

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_e2_1):                                         # using 80 files 

  # opening the file in read only mode
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction
  for j in range(len(df['neutrino']['interaction'])):

    # this is for electron neutrinos, between 4-7
    if df['neutrino']['interaction'][j] >=4 and df['neutrino']['interaction'][j] <= 7:

      model = df['cvnmap'][j].reshape((2,100,80))                         # reshaping the images
      e2_input_1.append(model[0])                                         # apending the x-z view to the corresponding array
      e2_input_2.append(model[1])                                         # apending the y-z view to the corresponding array
      e2_labels.append(int(2))                                            # appending a 2 to the labels for electron neutrinos 

    else:
      pass

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_e2_10):

  # opening the file in read only mode
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction 
  for i in range(len(df['neutrino']['interaction'])):

    if df['neutrino']['interaction'][i] > 7:  

      model = df['cvnmap'][i].reshape((2,100,80))                         # reshaping the images
      e2_input_1.append(model[0])                                         # apending the x-z view to the corresponding array
      e2_input_2.append(model[1])                                         # apending the y-z view to the corresponding array
      e2_labels.append(int(0))                                            # appending a 0 to the labels for non-muon and non-electron neutrinos 

    else:
      pass

In [ ]:
## PLOTTING THE LABELS TO CHECK IF IT IS BALANCED 

plt.hist(e2_labels)
plt.title('Labels for flavour classification')

In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# shuffling the labels and images 
e2_labels, e2_input_1, e2_input_2 = shuffle_data(e2_labels, e2_input_1, e2_input_2)

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
e2_input_1 = tf.expand_dims(e2_input_1, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
e2_input_2 = tf.expand_dims(e2_input_2, axis = 3)

# preparing the training, testing and validation data
train_input_1_e2, test_input_1_e2, val_input_1_e2, train_input_2_e2, test_input_2_e2, val_input_2_e2, train_labels_e2, val_labels_e2, test_labels_e2 = data(e2_input_1, e2_input_2, e2_labels, 0.8, 0.8)

# to be able to use categorical cross entropy 
train_labels_e2 = tf.keras.utils.to_categorical(train_labels_e2, 3)
test_labels_e2 = tf.keras.utils.to_categorical(test_labels_e2, 3)
val_labels_e2 = tf.keras.utils.to_categorical(val_labels_e2, 3)

This isn't a binary classification so the optimizer has to be changed to categorical crossentropy and in the last dense layer there has to be 3 nodes. 

In [ ]:
# Creates the CNN model for the x-z view of the neutrino interaction.
xz_input = Input(shape=np.shape(e2_input_1)[1:])
xz_model = create_convolution_layers(xz_input, np.shape(e2_input_1)[1:])

# Creates the CNN model for the y-z view of the neutrino interaction.
yz_input = Input(shape=np.shape(e2_input_1)[1:])
yz_model = create_convolution_layers(yz_input, np.shape(e2_input_1)[1:])

# Concatenates the two models 
conv = concatenate([xz_model, yz_model])

# Flattens the concatenated models 
conv = Flatten()(conv)

# Dense layers connect all the neurons of one layer to the ones in the next layer 
dense = Dense(8,                                                              # 64 is the dimensionality of the output space
              activation = "relu")(conv)                                      # Rectified Linear Unit as activation function
dense = Dropout(0.5)(dense)                                                   # 50% of the data is randomly excluded 

output = Dense(3,                                                             # 1 is the dimensionality of the output space 
                activation ="sigmoid")(dense)                                  # sigmoid as activation function. 
                                                                              # The input to the function is transformed into a value between 0.0 and 1.0.

# creates the model using the two model inputs 
model = Model(inputs = [xz_input, yz_input], outputs = [output])

# compiles the final model
model.compile(loss=tf.keras.losses.categorical_crossentropy,                       # using binary crossentropy as a loss function as we are doing a binary classification
# adam is used for optimization algorithm for stochastic gradient descent for training deep learning models
              optimizer='adam',                                               
              metrics=['accuracy'])                                           # using the accuracy as a metric to evaluate the model 

model.summary()

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history_elec = model.fit(x=[train_input_1_e2,train_input_2_e2],
                    y=train_labels_e2, batch_size=64,                              # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_input_1_e2,val_input_2_e2],val_labels_e2),
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

In [ ]:
## EVALUATING THE ACCURACY AND LOSS OF THE MODEL 

test_loss, test_acc = model.evaluate([test_input_1_e2,test_input_2_e2],  test_labels_e2, verbose=2)

In [ ]:
## PLOTTING THE ACCURACY OF THE MODEL

plt.plot(history_elec.history['accuracy'], label='accuracy')
plt.plot(history_elec.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid()
plt.title('Accuracy of neutrino flavour classifier')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## PLOTTING THE LOSS OF THE MODEL

plt.plot(history_elec.history['loss'], label='loss')
plt.plot(history_elec.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid()
plt.title('Loss of neutrino flavour classifier')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

<a name="extension3"></a>

## 7. Extension 3: Machine learning algorithm to determine $y=$ lepton energy over neutrino energy

This is similar to extension 1 but not we have to need an algorithm to determine lepton energy over neutrino energy. This is again a regression task. 

As this is a division, the first step is to find if there is any interaction that has lepton energy or neutrino energy that is equal to 0. For this we will use the 200 files. 


In [ ]:
## RETRIEVING THE FILES FOR THIS EXTENSION 
files_e3=data_retriever(200)

In [ ]:
## FINDING 0 LEPTON AND NEUTRINO ENERGY  

lep=[]                                                               # empty array to store the number of the interaction with 0 lepton energy
nu=[]                                                                # empty array to store the number of the interaction with 0 neutrino energy

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_e3):

  # opening the file in read only mode
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction 
  for i in range(len(df['cvnmap'])):

    # if the lepton energy is 0 it appends the number of the interaction to the array
    if df['neutrino']['lepenergy'][i] == 0:
      lep.append(df['neutrino']['interaction'][i])

    else:
      pass
  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction 
  for i in range(len(df['cvnmap'])):

    # if the lepton energy is 0 it appends the number of the interaction to the array
    if df['neutrino']['nuenergy'][i] == 0:
      nu.append(df['neutrino']['interaction'][i])

    else:
      pass

There is 0 lepton and neutrino energy for interaction number 15. These need to be excluded from the code. 

Now that this has been identified we can proceed with the task. 

In [ ]:
## RETRIEVING THE NECESSARY FILES 

files_e3_2=data_retriever(5)

In [ ]:
# LABELS AND IMAGES INPUT FOR MODEL 

y_energy_labels = []                                                      # empty array for values of lepton energy over neutrino energy 
y_energy_input_1=[]                                                       # empty array for the x-z view of the neutrino interaction
y_energy_input_2=[]                                                       # empty array for the y-z view of the neutrino interaction

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_e3_2):

  # opening the file in read only mode
  df=h5py.File(filename, 'r')

  # iterating over the length of the images to obtain the labels and input images for each interaction 
  for i in range(len(df['cvnmap'])):

    # excluding interaction 15 from the code 
    if df['neutrino']['interaction'][i] < 15 or df['neutrino']['interaction'][i] == 16:
      
      # calculating lepton energy over neutrino energy 
      y = df['neutrino']['lepenergy'][i] / df['neutrino']['nuenergy'][i] 
      
      y_energy_labels.append(y)                                           # apending y to the label array
      model = df['cvnmap'][i].reshape((2,100,80))                         # reshaping the images
      y_energy_input_1.append(model[0])                                   # apending the x-z view to the corresponding array
      y_energy_input_2.append(model[1])                                   # apending the y-z view to the corresponding array
    else:
      pass

In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
y_energy_input_1 = tf.expand_dims(y_energy_input_1, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
y_energy_input_2 = tf.expand_dims(y_energy_input_2, axis = 3)

# normalizing the energy labels 
y_energy_labels=y_energy_labels/np.max(y_energy_labels)

# preparing the training, testing and validation data
train_input_1_y_energy, test_input_1_y_energy, val_input_1_y_energy, train_input_2_y_energy, test_input_2_y_energy, val_input_2_y_energy, train_labels_y_energy, val_labels_y_energy, test_labels_y_energy = data(y_energy_input_1, y_energy_input_2, y_energy_labels, 0.8, 0.8)

In [ ]:
## CREATING THE MODEL FOR LEPTON ENERGY OVER NEUTRINO ENERGY DETERMINATION 

model = model_regression(y_energy_input_1)

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history_E3 = model.fit(x=[train_input_1_y_energy,train_input_2_y_energy],
                    y=train_labels_y_energy, batch_size=32,                        # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_input_1_y_energy,val_input_2_y_energy],
                                     val_labels_y_energy),
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

In [ ]:
## EVALUATING THE MAE AND MSE OF THE MODEL 

test_loss, test_mae, test_mse = model.evaluate([test_input_1_y_energy,test_input_2_y_energy],  test_labels_y_energy, verbose=2)

In [ ]:
## PLOTTING THE MSE OF THE MODEL 

plt.plot(history_E3.history['mse'], label='mean squared error')
plt.plot(history_E3.history['val_mse'], label = 'validation mean squared error')
plt.xlabel('Epoch')
plt.ylabel('mean squared error')
plt.legend(loc='upper right')
plt.grid()
plt.title('MSE of the model to determine \n lepton energy over neutrino energy')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## PLOTTING THE MAE OF THE MODEL 

plt.plot(history_E3.history['mae'], label='mean average error')
plt.plot(history_E3.history['val_mae'], label = 'validation mean average error')
plt.xlabel('Epoch')
plt.ylabel('mean average error')
plt.legend(loc='upper right')
plt.grid()
plt.title('MAE of the model to determine \n lepton energy over neutrino energy')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

<a name="extension5"></a>

## 8. Extension 5: Machine learning algorithm to determine the interaction mode

In this section we will be developing an algorithm to classify the neutrino interaction model. This is another classification task with five groups in this case, the same procedure as extension 2 will be used. The different interaction modes and the corresponding labels are shown below: 

- Label 0 for charged-current quasi-elastic events 

- Label 1 for charged-current resonant events 

- Label 2 for charged-current deep-inelastic scattering 

- Label 3 for charged-current other events 

- Label 4 for neutral charged current events 

In [ ]:
## RETRIEVING THE NECESSARY FILES FOR THIS EXTENSION 

files_5=data_retriever(20)

In [ ]:
## OBTAINING THE LABELS FOR THE INTERACTION MODE 

int_mode_labels=[]                   # empty array for the lables corresponding to the interaction mode 

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_5):

  # opening the file in read only mode
  df=h5py.File(filename)
               
  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction 
  for i in range(len(df['neutrino']['interaction'])):

    if (df['neutrino']['interaction'][i]) == 0 or (df['neutrino']['interaction'][i]) == 4 or (df['neutrino']['interaction'][i]) == 8:
      int_mode_labels.append(int(0)) # appending 0 for CC QE events

    elif (df['neutrino']['interaction'][i]) == 1 or (df['neutrino']['interaction'][i]) == 5 or (df['neutrino']['interaction'][i]) == 9:
      int_mode_labels.append(int(1)) # appending 1 for CC RES events  
    
    elif (df['neutrino']['interaction'][i]) == 2 or (df['neutrino']['interaction'][i]) == 6 or (df['neutrino']['interaction'][i]) == 10:
      int_mode_labels.append(int(2)) # appending 2 for CC DIS events  

    elif (df['neutrino']['interaction'][i]) == 3 or (df['neutrino']['interaction'][i]) == 7 or (df['neutrino']['interaction'][i]) == 11:
      int_mode_labels.append(int(3)) # appending 3 for CC other events 

    elif (df['neutrino']['interaction'][i]) == 12 or (df['neutrino']['interaction'][i]) == 13:
      int_mode_labels.append(int(4)) # appending 4 for NC events

    else:
      pass 

Now that we have the labels for the interaction mode, we need to make sure the labels are balanced. We will check the balance by plotting the labels on a histogram. 

In [ ]:
## CHECKING IF THE LABELS ARE BALANCED 

plt.hist(int_mode_labels)
plt.title('Imbalanced labels for neutrino interaction mode')

It can clearly be seen that the labels are not balanced. In order to balance these, we will use the same method we have been using for the other tasks. We will loop over more files for the label with least events and over less files for the labels with more events. This is a more complex balance to achieve as we are dealing with 5 interaction modes. 

In [ ]:
## SPLITTING THE FILES TO ITERATE OVER IN ORDER TO BALANCE THE LABELS

files_5_80, files_5_20 = train_test_split(files_5, train_size = 0.8, shuffle = False)
files_5_95, files_5_05 = train_test_split(files_5, train_size = 0.95, shuffle = False)
files_5_75, files_5_25 = train_test_split(files_5, train_size = 0.75, shuffle = False)

In [ ]:
## GETTING THE LABELS FOR THE INTERACTION MODE CLASSIFIER AND BALANCING THEM 

int_mode_labels_b=[]                                                # empty array to store the labels of the interaction mode
int_mode_input_1=[]                                                 # empty array for the x-z view of the neutrino interaction
int_mode_input_2=[]                                                 # empty array for the y-z view of the neutrino interaction

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_5_25):                                   # using 0-4 files 

  # opening the file in read only mode
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction 
  for i in range(len(df['neutrino']['interaction'])):

    # if the neutrino interaction number is 0, 4 or 8, the events is a CC QE event 
    if (df['neutrino']['interaction'][i]) == 0 or (df['neutrino']['interaction'][i]) == 4 or (df['neutrino']['interaction'][i]) == 8:

      model = df['cvnmap'][i].reshape((2,100,80))                  # reshaping the images
      int_mode_input_1.append(model[0])                            # apending the x-z view to the corresponding array
      int_mode_input_2.append(model[1])                            # apending the y-z view to the corresponding array
      int_mode_labels_b.append(int(0))                             # appending 0 for CC QE events

    else:
      pass

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_5_20):                                  # using 0-3 files 

  # opening the file in read only mode
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction
  for i in range(len(df['neutrino']['interaction'])):

    # if the neutrino interaction number is 1, 5 or 9, the events is a CC RES event 
    if (df['neutrino']['interaction'][i]) == 1 or (df['neutrino']['interaction'][i]) == 5 or (df['neutrino']['interaction'][i]) == 9:

      int_mode_labels_b.append(int(1))                             # appending 1 for CC RES events 
      model = df['cvnmap'][i].reshape((2,100,80))                  # reshaping the images
      int_mode_input_1.append(model[0])                            # apending the x-z view to the corresponding array
      int_mode_input_2.append(model[1])                            # apending the y-z view to the corresponding array

    else:
      pass

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_5_05):                                  # using 0-1 files 

  # opening the file in read only mode
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction
  for i in range(len(df['neutrino']['interaction'])): 
    
    # if the neutrino interaction number is 2, 6 or 10, the events is a CC DIS event 
    if (df['neutrino']['interaction'][i]) == 2 or (df['neutrino']['interaction'][i]) == 6 or (df['neutrino']['interaction'][i]) == 10:

      int_mode_labels_b.append(int(2))                             # appending 2 for CC DIS events  
      model = df['cvnmap'][i].reshape((2,100,80))                  # reshaping the images
      int_mode_input_1.append(model[0])                            # apending the x-z view to the corresponding array
      int_mode_input_2.append(model[1])                            # apending the y-z view to the corresponding array

    else:
      pass

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_5):                                     # using 0-15 files 

  # opening the file in read only mode
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction
  for i in range(len(df['neutrino']['interaction'])):

    # if the neutrino interaction number is 3, 7 or 11, the events is a CC other event
    if (df['neutrino']['interaction'][i]) == 3 or (df['neutrino']['interaction'][i]) == 7 or (df['neutrino']['interaction'][i]) == 11:

      int_mode_labels_b.append(int(3))                             # appending 3 for CC other events 
      model = df['cvnmap'][i].reshape((2,100,80))                  # reshaping the images
      int_mode_input_1.append(model[0])                            # apending the x-z view to the corresponding array
      int_mode_input_2.append(model[1])                            # apending the y-z view to the corresponding array

    else:
      pass

# iterating over the array of file names to obtain the lables and the input images for each interaction
for filename in tqdm(files_5_25): # using 0-4 files

  # opening the file in read only mode
  df=h5py.File(filename, 'r')

  # iterating over the length of neutrino interactions to obtain the labels and input images for each interaction
  for i in range(len(df['neutrino']['interaction'])):

    # if the neutrino interaction number is 12 or 13, the events is a NC other event
    if (df['neutrino']['interaction'][i]) == 12 or (df['neutrino']['interaction'][i]) == 13:

      int_mode_labels_b.append(int(4))                             # appending 4 for NC events
      model = df['cvnmap'][i].reshape((2,100,80))                  # reshaping the images
      int_mode_input_1.append(model[0])                            # apending the x-z view to the corresponding array
      int_mode_input_2.append(model[1])                            # apending the y-z view to the corresponding array
      
    else:
      pass 

In [ ]:
## CHECKING IF THE LABELS ARE BALANCED 

plt.hist(int_mode_labels_b)
plt.title('Balanced labels for neutrino interaction mode')

The labels are not perfectly balanced but they are a lot more balanced than the initial labels. 

In [ ]:
## PREPARING THE DATA TO FEED INTO THE NEURAL NETWORK

# shuffling the labels and images 
int_mode_labels_b, int_mode_input_1, int_mode_input_2 = shuffle_data(int_mode_labels_b, int_mode_input_1, int_mode_input_2)

# expanding the dimensions of the images corresponding to the x-z view of the neutrino interaction
int_mode_input_1 = tf.expand_dims(int_mode_input_1, axis = 3)

# expanding the dimensions of the images corresponding to the y-z view of the neutrino interaction
int_mode_input_2 = tf.expand_dims(int_mode_input_2, axis = 3)

# preparing the training, testing and validation data
train_int_mode_input_1, test_int_mode_input_1, val_int_mode_input_1, train_int_mode_input_2, test_int_mode_input_2, val_int_mode_input_2, train_int_mode_labels_b, val_int_mode_labels_b, test_int_mode_labels_b = data(int_mode_input_1, int_mode_input_2, int_mode_labels_b, 0.8, 0.8)

# to be able to use categorical cross entropy 
train_int_mode_labels_b = tf.keras.utils.to_categorical(train_int_mode_labels_b, 5)
test_int_mode_labels_b = tf.keras.utils.to_categorical(test_int_mode_labels_b, 5)
val_int_mode_labels_b = tf.keras.utils.to_categorical(val_int_mode_labels_b, 5)

We won't be using the definition to create the model as we need to change the number of nodes in the last dense layer to be 5, as well as the loss, which has to be categorical cross entropy. 

In [ ]:
# Creates the CNN model for the x-z view of the neutrino interaction.
xz_input = Input(shape=np.shape(int_mode_input_1)[1:])
xz_model = create_convolution_layers(xz_input, np.shape(int_mode_input_1)[1:])

# Creates the CNN model for the y-z view of the neutrino interaction.
yz_input = Input(shape=np.shape(int_mode_input_1)[1:])
yz_model = create_convolution_layers(yz_input, np.shape(int_mode_input_1)[1:])

# Concatenates the two models 
conv = concatenate([xz_model, yz_model])

# Flattens the concatenated models 
conv = Flatten()(conv)

# Dense layers connect all the neurons of one layer to the ones in the next layer 
dense = Dense(8,                                                              # 64 is the dimensionality of the output space
              activation = "relu")(conv)                                      # Rectified Linear Unit as activation function
dense = Dropout(0.5)(dense)                                                   # 50% of the data is randomly excluded 

output = Dense(5,                                                             # 1 is the dimensionality of the output space 
                activation ="sigmoid")(dense)                                  # sigmoid as activation function. 
                                                                              # The input to the function is transformed into a value between 0.0 and 1.0.

# creates the model using the two model inputs 
model = Model(inputs = [xz_input, yz_input], outputs = [output])

# compiles the final model
model.compile(loss=tf.keras.losses.categorical_crossentropy,                       # using binary crossentropy as a loss function as we are doing a binary classification
# adam is used for optimization algorithm for stochastic gradient descent for training deep learning models
              optimizer='adam',                                               
              metrics=['accuracy'])                                           # using the accuracy as a metric to evaluate the model 

model.summary()

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",                       # quantity to be monitored
                                        mode ="min",                               # training will stop when val_loss is at its minimum
                                        patience = 5,                              # number of epochs with no improvement after which training will be stopped
                                        restore_best_weights = True)               # restore to the val_loss of the first epoch from which there is no improvement 

history_elec = model.fit(x=[train_int_mode_input_1,train_int_mode_input_2],
                    y=train_int_mode_labels_b, batch_size=64,                      # number of training examples utilized in one iteration
                    epochs=50,                                                     # 1 epoch: when the entire dataset is passed forward and backward through the CNN
                    validation_data=([val_int_mode_input_1,val_int_mode_input_2],val_int_mode_labels_b),
                    callbacks =[earlystopping] )                                   # used to avoid overtraining

In [ ]:
## EVALUATING THE ACCURACY AND LOSS OF THE MODEL 

test_loss, test_acc = model.evaluate([test_int_mode_input_1,test_int_mode_input_2],  test_int_mode_labels_b, verbose=2)

In [ ]:
## PLOTTING THE ACCURACY OF THE MODEL

plt.plot(history_elec.history['accuracy'], label='accuracy')
plt.plot(history_elec.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid()
plt.title('Accuracy of neutrino interaction mode classifier')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

In [ ]:
## PLOTTING THE LOSS OF THE MODEL

plt.plot(history_elec.history['loss'], label='loss')
plt.plot(history_elec.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid()
plt.title('Loss of neutrino interaction mode classifier')
#f.savefig("dataset.png")                # saves the figure to be able to download it to a desktop 
#files.download("dataset.png")           # saves the figure to the desktop and sets the name for it

The accuracy of the classifier is very low which could be due to the computational limitations of the task, as the RAM crashed for if the network operated on a larger dataset. This could be repeated with a more complex network operating on a larger dataset to improve the improve the performance. 

<a name="conc"></a>

## 9. Conclusion

A multi-view CNN was used to perform different tasks on a dataset containing information on neutrino
interactions. The first objective was to develop a machine learning algorithm to classify muon neutrinos. The
initial classifier achieved an accuracy 82.50% and a loss of 0.4116. It was found that the dataset being used to
train this classifier contained approximately 50% of DIS events and only 13% QE events. The accuracy and the
loss improved by 3.2% and 10.6% respectively when the classifier was operated on a dataset of equal QE and
DIS events. The accuracy and the loss improved even more, by 11.2% and 43.3% respectively when it operated
on a dataset uniquely containing QE events. When the classifier was tested on datasets of high and low energy
muons and neutrinos, no significant change in performance was observed. This demonstrated the that the
performance of the model is dependent on those variables that affect the images. The importance of having
a balanced dataset was explored, in the contrary case false positives were obtained. Lastly, the model was
used to classify neutrino flavours and interaction modes and to detect neutrino energies and lepton energy
over neutrino energy. Overall, the results obtained for all the tasks are acceptable but were constrained by
computational limitations. The results can be improved using a larger dataset, which is only possible with a
more powerful GPU and more RAM space. 

<a name="ref"></a>

## 10. References

[1] E. Nurse, A. Korn. PHAS0040 Nuclear and Particle Physics, University College London (UCL). pp 5-6, 2020.


[2] Emsley, John. Nature's Building Blocks ((Hardcover, First Edition) ed.). Oxford University Press. pp. 521–22. ISBN 978-
0-19-850340-8, 2001.


[3] R. Oerter. The Theory of Almost Everything: The Standard Model, the Unsung Triumph of Modern Physics (Kindle ed.).
Penguin Group. p. 2. ISBN 978-0-13-236678-6, 2006.


[4] Butterworth, J. The Standard Model: how far can it go and how can we tell?. Philosophical Transactions of the Royal
Society A: Mathematical, Physical and Engineering Sciences, 374(2075), p.20150260, 2016


[5] Rosario Rocco, D. Muon Neutrino Disappearance in NOvA with a Deep Convolutional Neural Network Classifier, 2016


[6] H. Becquerel. Compt. Rend., 1896.


[7] W. Pauli. Pauli Letter Collection, CERN, 1930.


[8] J. Chadwick. The Existence of a Neutron. Royal Society of London Proceedings Series A, 136:692–708, June 1932.


[9] E. Fermi. Versuch einer Theorie der β-Strahlen. I. Zeitschrift fur Physik, 88:161– 177, March 1934.


[10] C. L. Cowan Jr., F. Reines, F. B. Harrison, H. W. Kruse, and A. D. McGuire. Detection of the free neutrino: A
confirmation. Science, 124(3212):pp. 103–104, 1956.


[11] E. Vitagliano, J. Redondo, G. Raffelta et al. Solar neutrino flux at keV energies. 2017.


[12] Z. Szadowski, D. Glas, K. Pytel et al. Artificial Neural Networks as a FPGA Trigger for a Detection of NeutrinoInduced Air Showers. 2016.


[13] Z. Maki, M. Nakagawa, S. Sakata et al. Remarks on the Unified Model of Elementary Particles. 1962.


[14] F. Halzen and A.D. Martin. Quarks and leptons: an introductory course in modern particle physics. Wiley, 1984.


[15] Q. Ho-Kim and X.Y. Pham. Elementary Particles and Their Interactions: Concepts and Phenomena. Springer Berlin
Heidelberg, 2010.


[16] Stuart Charles Fuess. Neutrino-proton and anti-neutrino-proton elastic scattering. Technical report, Illinois Univ., Urbana, IL (United States), 1981.


[17] C.H. Llewellyn Smith. Neutrino reactions at accelerator energies. Physics Reports, 3(5):261 – 379, 1972.


[18] D. Rein, L. M. Sehgal. Neutrino-excitation of baryon resonances and single pion production. Annals of Physics,
133(1):79–153, 1981.


[19] MAG Aivazis, Fredrick I Olness, and Wu-Ki Tung. Leptoproduction of heavy quarks. i. general formalism and
kinematics of charged current and neutral current production processes. Physical Review D, 50(5):3085, 1994.


[20] Y. Fukuda , T.Hayakawa , E.Ichihara et al. Evidence for oscillation of atmospheric neutrinos. 1998.
18014458, January 2021


[21] F. Capozzi, E. Lisi and A. Marrone et al. Neutrino masses and mixings: Status of known and unknown 3ν parameters.
2016.


[22] J. Sonneveld Searches for physics beyond the standard model at the LHC. 2018.


[23] G.N Perdue, A. Ghosh, M. Wospakrik et al. Reducing model bias in a deep learning classifier using domain
adversarial neural networks in MINERνA experiment. 2018.


[24] M. Brenzke Development and Optimization of Deep Neural Networks for Energy Reconstruction of Muon Events in
IceCube. 2017.


[25] Y. Fukud, T. Hayakawa, E. Ich. et al. The Super-Kamiokande detector. 2002.


[26] R. Acciarri, M. A. Acero, M. Adamowski et al. Long-Baseline Neutrino Facility (LBNF) and Deep Underground
Neutrino Experiment (DUNE) Conceptual Design Report Volume 1: The LBNF and DUNE Projects. 2016.


[27] A. Aurisano, A. Radovic, D. Rocco et al. A Convolutional Neural Network Neutrino Event Classifier. 2016.


[28] P Adamson, K Anderson, M Andrews, R Andrews, I Anghel, D Augustine, A Aurisano, S Avvakumov, DS Ayres, B
Baller, et al. The numi neutrino beam. Nuclear Instruments and Methods in Physics Research Section A: Accelerators,
Spectrometers, Detectors and Associated Equipment, 806:279–306, 2016.


[29] Y. LeCun, B. Boser, J. S. Denker, D. Henderson, R. E. Howard, W. Hubbard et al., Backpropagation applied to
handwritten zip code recognition, Neural Comput. 1 (Dec., 1989) 541–551.


[30] Y. Guo et al. Deep learning for visual understanding: A review. Neurocomputing, 187, 11 2015.


[31] K. O’Shea and Ryan Nash. An introduction to convolutional neural networks, 2015.


[32] Team, K.. Keras Documentation: The Functional API. [online] Keras.io. Available at:
https://keras.io/guides/functional_api/#multi-input-and-multi-output-models, 2021 [Accessed 01 January 2021]. 

[33] Liu, S. and Deng, W. Very deep convolutional neural network-based image classification using small training sample
size. 2015 3rd IAPR Asian Conference on Pattern Recognition (ACPR), 2015.


[34] He, K., Zhang, X., Ren, S. and Sun, J. Deep Residual Learning for Image Recognition. 2016 IEEE Conference on
Computer Vision and Pattern Recognition (CVPR), 2016.


[35] Uddin, M. Addressing Accuracy Paradox Using Enhanched Weighted Performance Metric in Machine Learning. 2019
Sixth HCT Information Technology Trends (ITT), 2019.


[36] H.Sekiya J. Phys.: Conf. Ser. 718 062052, 2016